# **(1.0)** Reversible First Order Kinetics

The first order kinetics we were working with in the last lesson assumed that the reactions involved were **irreversible** - that is, the reaction **S &#8594; A** never ran in the opposite direction, **A &#8594; S**. In reality though, all reactions are reversible, even if the back-reactions are neglegible in some circumstances. To add an additional layer of realism, let's consider what reversible first order kinetics might look like. Instead of the straight, unidirectional lines from **S** to **P** we had last time, now we have:

![alt text](ReversibleFirstOrderDiagram.png "Title")

Notice that some of our reactions now have complementary reverse reactions running in the opposite directions. Here, we're using **Jab** to represent the net flux from S to A, where that net flux is the difference between the rate of the forward reaction **V2** and the rate of the reverse reaction **V-2**, and likewise for **Jbp**. **Jsa** is somewhat simpler since we're treating that as an irreversible reaction like before. Like before, each reaction rate **V1** through **V3** (and their reverse reactions, where appropriate) will have a corresponding rate constant and be proportional to substrate concentration. To put that mathematically:

$$ V_1 = k_1[S] $$
$$ V_2 = k_2[A] $$
$$ V_{-2} = k_{-2}[B] $$
$$ V_3 = k_3[B] $$
$$ V_{-3} = k_{-3}[P] $$

Because we have both these forward and reverse reactions, that means we must consider both when figuring out the net change in concentration of a given metabolite pool. The resulting differential equations we get are:

### Labeled
$$ \frac{dLA}{dt} = f_s(V_1) - f_A(V_2 - V_{-2}) $$

$$ \frac{dLB}{dt} = f_a(V_2 - V_{-2}) - f_B(V_3 - V_{-3}) $$

$$ \frac{dLP}{dt} = f_B(V_3 - V_{-3}) $$

### Unlabeled
$$ \frac{dA}{dt} = (1-f_S)(V_1) - (1-f_A)(V_2 - V_{-2}) $$

$$ \frac{dB}{dt} = (1-f_A)(V_2 - V_{-2}) - (1-f_B)(V_3 - V_{-3}) $$

$$ \frac{dP}{dt} = (1-f_B)(V_3 - V_{-3}) $$

The actual logic of our simulation will, of course, remain the same, except for the inclusion of code to calculate the relevant reverse reactions and incorporate them into our differential equation calculations.

<div class="alert alert-block alert-info">

**Instructions:** Run the three code blocks below; the first defines the functions we'll need to execute the simulation and the second opens up an interactive widget for you to play around with it. See how this simulation is similar and different from the ones we ran before. What effect does the inclusion of reverse reactions have? 
    
</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell, we are importing the Python packages we'll need for today's exercises. 
#-----------------------------------------------------------------------------------------------------------------------

import numpy as np                  
import math                                                                      
import seaborn as sns               
import matplotlib.pyplot as plt     
import ipywidgets as widgets        
from IPython.display import display, Markdown, clear_output             
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots 
import pandas as pd                 
from scipy import interpolate       
%matplotlib inline                 
sns.set()                           

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell we are defining functions that we will need during our simulation, as well as the function that carries
# out the simulation itself. Comments have been added to highlight changes to account for reversibility.
#-----------------------------------------------------------------------------------------------------------------------

def evaluate_flux(k, S):                                           
    flux = k*S                                                                                                             
    return flux                                                   


def evaluate_dLAdt_f(Jsa, Jab, f_s, f_a):                                       # We are evaluating the change in labeled and unlabeled
                                                                                # metabolite concentrations as a result of forward and
                                                                                # reverse fluxes in this simulation independently
    '''                                                                         
    DESCRIPTION
    
    Takes the fluxes into out and out of metabolite A (Jsa and Jab)
    in the forward directionand the fractional labeling of S and A 
    to calculate the change in concentration of either labeled or unlabeled A.
    
    INPUTS
    
    Jsa = the numeric flux value Jsa (flux from S to A)
    
    Jab = the numeric flux value Jab (flux from A to B)
    
    f_s = a float value from 0 to 1 representing the fractional labeling of S
    
    f_a = a float value from 0 to 1 representing the fractional labeling of A
    
    OUTPUTS
    
    dLAdt = the change in the labeled or unlabeled concentration
    of A
    
    '''
    
    dLAdt = Jsa * f_s - Jab * f_a
    return dLAdt

def evaluate_dLAdt_b(Jab_b, f_a, f_b):
    dLAdt = Jab_b * f_b
    return dLAdt

def evaluate_dLBdt_f(Jab, Jbp, f_a, f_b):
    dLBdt = Jab * f_a - Jbp * f_b
    return dLBdt

def evaluate_dLBdt_b(Jab_b, Jbp_b, f_b, f_p):
    dLBdt = - Jab_b * f_b + Jbp_b * f_p
    return dLBdt

def evaluate_dLPdt_f(Jbp, f_b):
    dLPdt = Jbp * f_b
    return dLPdt

def evaluate_dLPdt_b(Jbp_b, f_p):
    dLPdt = - Jbp_b * f_p
    return dLPdt

def evaluate_fractional_labeling(current_labeled, current_total):      
    if current_total == 0:                                             
        f =  0                                                         
    else:                                                             
        f = current_labeled / current_total                                                                                     
    return f

#-----------------------------------------------------------------------------------------------------------------------
# Now we define our main simulation function. Comments are used to highlight where the code has been changed to account
# for reaction reversibility
#-----------------------------------------------------------------------------------------------------------------------

def reversible_kinetic_simulation(total_time = 1000, timestep = 1, switch_point = 0, S = 10, A = 0, B = 0,P = 0, LS = 0, LA = 0, LB = 0,
        LP = 0, S_switch = 10, LS_switch = 0, K_SA = 0.1, K_AB = 0.1, K_AB_b = 0.01,
        K_BP = 0.1, K_BP_b = 0.01, export = False, compare = 'None', filename = '', subset = False):
    
    '''
    DESCRIPTION
    
    A function that carries out a metabolic modeling simulation using reversible first-order kinetics.
    Accepts arguments to both export generated data and import another group's data to 
    compare with simulation results. Can also subset generated data and add random noise
    before exporting.
    
    INPUTS
    
    total_time = A numeric value with arbitrary units that determines how long to run
    the simulation for.
    
    timestep = A numeric value representing how much to increment the time value each
    loop of the simulation.
    
    switch_point = A numeric value representing at which point to switch from an initial
    concentration of labeled and unlabeled S to a "post-switch" value for these same
    quantities
    
    S = A numeric value representing the set concentration of unlabeled S pre-switch
    
    A = A numeric value representing the initial concentration of unlabeled A
    
    B = A numeric value representing the initial concentration of unlabeled B
    
    P = A numeric value representing the initial concentration of unlableed P
    
    LS = A numeric value representing the set concentration of labeled S pre-switch
    
    LA = A numeric value representing the initial concentration of labeled A
    
    LB = A numeric value representing the initial concentration of labeled B
    
    LP = A numeric value representing the initial concentration of labeled P
    
    S_switch = A numeric value representing the set concentration of unlabeled 
    S post-switch
    
    LS_switch = A numeric value representing the set concentration of labeled 
    S post-switch    
    
    k_SA = A numeric value representing the first-order rate constant for the
    reaction S -> A
    
    k_AB = A numeric value represnting the first-order rate constant for the 
    reaction A -> B
    
    k_AB_b = A numeric value representing the first-order rate constant for the
    reaction B -> A
    
    k_BP = A numeric value representing the first-order rate constant for the 
    reaction B -> P
    
    k_BP_b = A numeric value representing the first-order rate constant for the
    reaction P -> B
    
    export = A boolean value that determines whether or not to export the 
    simulated data as a .csv file
    
    compare = A boolean value that determines whether or not to load in
    another group's data from a .csv file and plot it out
    
    filename = A string corresponding to the name of the file loaded in
    if compare == True
    
    subset = A boolean value that determines whether or not to subset and
    add noise to the simulated data before exporting (only matters if 
    export == True)
    
    OUTPUTS
    
    None
    
    '''
    
    times = np.arange(0, total_time, timestep)                   
    
    S_total = S + LS                                              
    A_total = A + LA                                              
    B_total = B + LB
    P_total = P + LP
    
    S_list = [S]                                                                                                    
    A_list = [A]                                                                                                 
    B_list = [B]                                                  
    P_list = [P]                                                  
    
    LS_list = [LS]                                               
    LA_list = [LA]
    LB_list = [LB]
    LP_list = [LP]
        
    S_total_list = [S_total]                                      
    A_total_list = [A_total]
    B_total_list = [B_total]
    P_total_list = [P_total]
    
    fs_list = [evaluate_fractional_labeling(LS, S_total)]         
    fa_list = [evaluate_fractional_labeling(LA, A_total)]         
    fb_list = [evaluate_fractional_labeling(LB, B_total)]         
    fp_list = [evaluate_fractional_labeling(LP, P_total)]
    
    jsa_list = [evaluate_flux(S_total_list[-1], K_SA)]            # We track the forward and reverse fluxes independently
    jab_list = [evaluate_flux(A_total_list[-1], K_AB)]
    jab_b_list = [evaluate_flux(B_total_list[-1], K_AB_b)]
    jbp_list = [evaluate_flux(B_total_list[-1], K_BP)]
    jbp_b_list = [evaluate_flux(P_total_list[-1], K_BP_b)]
    
    for i in times:                                             
                                                            
        if i >= switch_point:
            current_S = S_switch
            current_LS = LS_switch
        else:
            current_S = S
            current_LS = LS
     
        #------------------------------------------------------------------------------------------------------------
        # Evaluating current forward and reverse fluxes, changes in labeled/unlabeled/total concentrations, and fractional labeling
        #------------------------------------------------------------------------------------------------------------
        
        current_Jsa = evaluate_flux(S_total_list[-1], K_SA)       # Evaluating all our fluxes, forward and reverse. Note that we are 
        current_Jab = evaluate_flux(A_total_list[-1], K_AB)       # evaluating them the appropriate forward and reverse rate
        current_Jab_b = evaluate_flux(B_total_list[-1], K_AB_b)   # constants
        current_Jbp = evaluate_flux(B_total_list[-1], K_BP)
        current_Jbp_b = evaluate_flux(P_total_list[-1], K_BP_b)

        # We first evaluate changes resulting from the forward fluxes in our system
        
        current_A = A_list[-1] + (evaluate_dLAdt_f(current_Jsa, current_Jab, (1 - fs_list[-1]), (1 - fa_list[-1])) * timestep)
        current_B = B_list[-1] + (evaluate_dLBdt_f(current_Jab, current_Jbp, (1 - fa_list[-1]), (1 - fb_list[-1])) * timestep)
        current_P = P_list[-1] + (evaluate_dLPdt_f(current_Jbp, (1-fb_list[-1])) * timestep)
        current_LA = LA_list[-1] + (evaluate_dLAdt_f(current_Jsa, current_Jab, fs_list[-1], fa_list[-1]) * timestep)
        current_LB = LB_list[-1] + (evaluate_dLBdt_f(current_Jab, current_Jbp, fa_list[-1], fb_list[-1]) * timestep)
        current_LP = LP_list[-1] + (evaluate_dLPdt_f(current_Jbp, fb_list[-1]) * timestep)
        
        # We update our totals and fractional labeling because we want the change in labeling state resulting
        # from the forward reactions to be reflected when we consider the reverse reactions
        
        current_S_total = current_S + current_LS
        current_A_total = current_A + current_LA
        current_B_total = current_B + current_LB
        current_P_total = current_P + current_LP        
        current_fs = evaluate_fractional_labeling(current_LS, current_S_total)
        current_fa = evaluate_fractional_labeling(current_LA, current_A_total)
        current_fb = evaluate_fractional_labeling(current_LB, current_B_total)
        current_fp = evaluate_fractional_labeling(current_LP, current_P_total)        
        
        # Next, we evaluate everything in the reverse direction
        
        current_A = current_A + (evaluate_dLAdt_b(current_Jab_b, (1 - fa_list[-1]), (1 - fb_list[-1])) * timestep)
        current_B = current_B + (evaluate_dLBdt_b(current_Jab_b, current_Jbp_b, (1 - fb_list[-1]), (1 - fp_list[-1])) * timestep)
        current_P = current_P + (evaluate_dLPdt_b(current_Jbp_b, (1 - fp_list[-1])) * timestep)
        current_LA = current_LA + (evaluate_dLAdt_b(current_Jab_b, fa_list[-1], fb_list[-1]) * timestep)
        current_LB = current_LB + (evaluate_dLBdt_b(current_Jab_b, current_Jbp_b, fb_list[-1], fp_list[-1]) * timestep)
        current_LP = current_LP + (evaluate_dLPdt_b(current_Jbp_b, fp_list[-1]) * timestep)
        current_S_total = current_S + current_LS
        current_A_total = current_A + current_LA
        current_B_total = current_B + current_LB
        current_P_total = current_P + current_LP 
        
        # And now we update our totals and fractional labeling once more.
        
        current_fs = evaluate_fractional_labeling(current_LS, current_S_total)
        current_fa = evaluate_fractional_labeling(current_LA, current_A_total)
        current_fb = evaluate_fractional_labeling(current_LB, current_B_total)
        current_fp = evaluate_fractional_labeling(current_LP, current_P_total)    

        #------------------------------------------------------------------------------------------------------------
        # Now that we've calculated our current concentrations, fluxes, and fractional labeling values, we can update
        # our lists with these most recent values. If the simulation hasn't ended, these will be used as the basis for
        # the next timestep's calculations.
        #------------------------------------------------------------------------------------------------------------
        
        LS_list.append(current_LS)                                  
        LA_list.append(current_LA)                                  
        LB_list.append(current_LB)
        LP_list.append(current_LP)

        S_list.append(current_S)
        A_list.append(current_A)
        B_list.append(current_B)
        P_list.append(current_P)

        S_total_list.append(current_S_total)
        A_total_list.append(current_A_total)
        B_total_list.append(current_B_total)
        P_total_list.append(current_P_total)

        fs_list.append(current_fs)
        fa_list.append(current_fa)
        fb_list.append(current_fb)
        fp_list.append(current_fp)
        
        jsa_list.append(current_Jsa)
        jab_list.append(current_Jab)
        jab_b_list.append(current_Jab_b)
        jbp_list.append(current_Jbp)
        jbp_b_list.append(current_Jbp_b)
    
    
    jsa_net = np.array(jsa_list)                                       # We calculated forward and reverse fluxes, but 
    jab_net = np.array(jab_list) - np.array(jab_b_list)                # we may also be interested in seeing the net fluxes,
    jbp_net = np.array(jbp_list) - np.array(jbp_b_list)                # so we retrieve those numbers by converting to numpy arrays
                                                                       # and taking the difference
    
    times_including_initial = np.insert(times, 0, times[0] - timestep)   
    
    if compare == 'None':                                          
                    
        fig, ax = plt.subplots(1, 4, figsize = (16, 5))                          
        
        ax[0].plot(times_including_initial, S_total_list, label = 'S')           
        ax[0].plot(times_including_initial, A_total_list, label = 'A')          
        ax[0].plot(times_including_initial, B_total_list, label = 'B')           
        ax[0].plot(times_including_initial, P_total_list, label = 'P')           
        ax[0].legend()                                                          
        ax[0].set_title('Total Concentration Over Time', fontweight = 'bold')    
        ax[0].set_xlabel('Time', fontweight = 'bold')                            
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')             

        ax[1].plot(times_including_initial, jsa_net, label = 'S -> A')           
        ax[1].plot(times_including_initial, jab_net, label = 'A -> B')           
        ax[1].plot(times_including_initial, jbp_net, label = 'B -> P')          
        ax[1].legend()                                                           
        ax[1].set_title('Fluxes', fontweight = 'bold')                           
        ax[1].set_xlabel('Time', fontweight = 'bold')                            
        ax[1].set_ylabel('Flux', fontweight = 'bold')                            
        
        ax[2].plot(times_including_initial, jsa_list, label = 'S -> A')          
        ax[2].plot(times_including_initial, jab_list, label = 'A -> B')          
        ax[2].plot(times_including_initial, jbp_list, label = 'B -> P')          
        ax[2].plot(times_including_initial, jab_b_list, label = 'B -> A')        
        ax[2].plot(times_including_initial, jbp_b_list, label = 'P -> B')       
        ax[2].legend()                                                           
        ax[2].set_title('Fluxes',fontweight='bold')                              
        ax[2].set_xlabel('Time',fontweight='bold')                               
        ax[2].set_ylabel('Flux',fontweight='bold')                               
        
        ax[3].plot(times_including_initial, fs_list, label = 'S')               
        ax[3].plot(times_including_initial, fa_list, label = 'A')                
        ax[3].plot(times_including_initial, fb_list, label = 'B')               
        ax[3].plot(times_including_initial, fp_list, label = 'P')                
        ax[3].legend()                                                          
        ax[3].set_title('Fraction Labeled Over Time', fontweight = 'bold')      
        ax[3].set_xlabel('Time', fontweight = 'bold')                            
        ax[3].set_ylabel('Total Concentration', fontweight = 'bold')            
    
    elif compare == 'Just Concentration':                                           
                
        other_data = pd.read_csv(filename)                                          
            
        fig, ax = plt.subplots(1, 1, figsize = (10, 10))                            

        ax.plot(times_including_initial, S_total_list, label = 'S')                     
        ax.plot(times_including_initial, A_total_list, label = 'A')                     
        ax.plot(times_including_initial, B_total_list, label = 'B')                     
        ax.plot(times_including_initial, P_total_list, label = 'P')                     
        ax.scatter(other_data['Times'], other_data['Total_S'], label = 'S Comp')        
        ax.scatter(other_data['Times'], other_data['Total_A'], label = 'A Comp')        
        ax.scatter(other_data['Times'], other_data['Total_B'], label = 'B Comp')       
        ax.scatter(other_data['Times'], other_data['Total_P'], label = 'P Comp')       
        ax.legend()                                                                     
        ax.set_title("Other Group's Concentration Over Time", fontweight = 'bold')     
        ax.set_xlabel('Time', fontweight = 'bold')                                     
        ax.set_ylabel('Total Concentration', fontweight = 'bold')                      
                
    elif compare == 'ConcentrationAndFractionalLabeling':                           
        
        other_data = pd.read_csv(filename)     
        
        fig, ax = plt.subplots(1, 2, figsize=(16, 10))                              
                
        ax[0].plot(times_including_initial, S_total_list, label = 'S')
        ax[0].plot(times_including_initial, A_total_list, label = 'A')
        ax[0].plot(times_including_initial, B_total_list, label = 'B')
        ax[0].plot(times_including_initial, P_total_list, label = 'P')  
        ax[0].scatter(other_data['Times'], other_data['Total_S'], label = 'S Comp')
        ax[0].scatter(other_data['Times'], other_data['Total_A'], label = 'A Comp')
        ax[0].scatter(other_data['Times'], other_data['Total_B'], label = 'B Comp')
        ax[0].scatter(other_data['Times'], other_data['Total_P'], label = 'P Comp')     
        ax[0].legend()
        ax[0].set_title("Other Group's Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')

        ax[1].plot(times_including_initial, fs_list, label = 'S')
        ax[1].plot(times_including_initial, fa_list, label = 'A')
        ax[1].plot(times_including_initial, fb_list, label = 'B')
        ax[1].plot(times_including_initial, fp_list, label = 'P') 
        ax[1].scatter(other_data['Times'], other_data['fs'], label = 'S Comp')
        ax[1].scatter(other_data['Times'], other_data['fa'], label = 'A Comp')
        ax[1].scatter(other_data['Times'], other_data['fb'], label = 'B Comp')
        ax[1].scatter(other_data['Times'], other_data['fp'], label = 'P Comp')     
        ax[1].legend()
        ax[1].set_title("Other Group's Fraction Labeled Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Total Concentration', fontweight = 'bold')
            
    elif compare == 'Everything':                                                   
        
        other_data = pd.read_csv(filename)     
        
        fig, ax = plt.subplots(1, 4, figsize=(24, 10))


        ax[0].plot(times_including_initial, S_total_list, label = 'S')
        ax[0].plot(times_including_initial, A_total_list, label = 'A')
        ax[0].plot(times_including_initial, B_total_list, label = 'B')
        ax[0].plot(times_including_initial, P_total_list, label = 'P')  
        ax[0].scatter(other_data['Times'], other_data['Total_S'], label = 'S Comp')
        ax[0].scatter(other_data['Times'], other_data['Total_A'], label = 'A Comp')
        ax[0].scatter(other_data['Times'], other_data['Total_B'], label = 'B Comp')
        ax[0].scatter(other_data['Times'], other_data['Total_P'], label = 'P Comp')     
        ax[0].legend()
        ax[0].set_title("Other Group's Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')

        ax[1].plot(times_including_initial, jsa_list, label = 'S -> A')
        ax[1].plot(times_including_initial, jab_list, label = 'A -> B')
        ax[1].plot(times_including_initial, jbp_list, label = 'B -> P')  
        ax[1].scatter(other_data['Times'], other_data['SA_Flux'], label = 'S -> A Comp')
        ax[1].scatter(other_data['Times'], other_data['AB_Flux'], label = 'A -> B Comp')
        ax[1].scatter(other_data['Times'], other_data['BP_Flux'], label = 'B -> P Comp')  
        ax[1].legend()
        ax[1].set_title("Other Group's Fluxes", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Flux', fontweight = 'bold')

        ax[2].plot(times_including_initial, jsa_list, label = 'S -> A')
        ax[2].plot(times_including_initial, jab_list, label = 'A -> B')
        ax[2].plot(times_including_initial, jbp_list, label = 'B -> P')
        ax[2].plot(times_including_initial, jab_b_list, label = 'B -> A')
        ax[2].plot(times_including_initial, jbp_b_list, label = 'P -> B')
        ax[2].scatter(other_data['Times'], other_data['SA_Flux'], label = 'S -> A')
        ax[2].scatter(other_data['Times'], other_data['AB_Flux'], label = 'A -> B')
        ax[2].scatter(other_data['Times'], other_data['BP_Flux'], label = 'B -> P')
        ax[2].scatter(other_data['Times'], other_data['SA_b_Flux'], label = 'A -> S')
        ax[2].scatter(other_data['Times'], other_data['AB_b_Flux'], label = 'B -> A')
        ax[2].scatter(other_data['Times'], other_data['BP_b_Flux'], label = 'P -> B')           
        ax[2].legend()
        ax[2].set_title('Fluxes', fontweight = 'bold')
        ax[2].set_xlabel('Time', fontweight = 'bold')
        ax[2].set_ylabel('Flux', fontweight = 'bold')

        ax[3].plot(times_including_initial, fs_list, label = 'S')
        ax[3].plot(times_including_initial, fa_list, label = 'A')
        ax[3].plot(times_including_initial, fb_list, label = 'B')
        ax[3].plot(times_including_initial, fp_list, label = 'P') 
        ax[3].scatter(other_data['Times'], other_data['fs'], label = 'S Comp')
        ax[3].scatter(other_data['Times'], other_data['fa'], label = 'A Comp')
        ax[3].scatter(other_data['Times'], other_data['fb'], label = 'B Comp')
        ax[3].scatter(other_data['Times'], other_data['fp'], label = 'P Comp')     
        ax[3].legend()
        ax[3].set_title("Other Group's Fraction Labeled Over Time", fontweight='bold')
        ax[3].set_xlabel('Time', fontweight = 'bold')
        ax[3].set_ylabel('Total Concentration', fontweight = 'bold')
    
    if export:                                                             
        
        if not subset:                                                     
            
            results_dictionary = {'Times': times_including_initial,        
                                  'Total_S': S_total_list,                 
                                  'Total_A': A_total_list,
                                  'Total_B': B_total_list,
                                  'Total_P': P_total_list,
                                  'SA_Flux': jsa_list,
                                  'AB_Flux': jab_list,
                                  'BP_Flux': jbp_list,
                                  'AB_b_Flux': jab_b_list,
                                  'BP_b_Flux': jbp_b_list,
                                  'fs': fs_list,
                                  'fa': fa_list,
                                  'fb': fb_list,
                                  'fp': fp_list}
            results_df = pd.DataFrame(results_dictionary)                  
            results_df.to_csv('GroupX_FirstOrderData.csv')                 
            
            print('Exported Data!')
            
        elif subset:                                                       
            
            times_numpy = np.array(times_including_initial)[0::100]        
            S_total_numpy = np.array(S_total_list)[0::100]                 
            A_total_numpy = np.array(A_total_list)[0::100]
            B_total_numpy = np.array(B_total_list)[0::100]
            P_total_numpy = np.array(P_total_list)[0::100]
            jsa_numpy = np.array(jsa_list)[0::100]
            jab_numpy = np.array(jab_list)[0::100]
            jbp_numpy = np.array(jbp_list)[0::100]
            jab_b_numpy = np.array(jab_b_list)[0::100]
            jbp_b_numpy = np.array(jbp_b_list)[0::100]
            fs_numpy = np.array(fs_list)[0::100]
            fa_numpy = np.array(fa_list)[0::100]
            fb_numpy = np.array(fb_list)[0::100]
            fp_numpy = np.array(fp_list)[0::100]
            
            S_total_noise = np.random.normal(0, 1, size = S_total_numpy.shape) 
            S_total_noise = S_total_noise * 0.1 * S_total_numpy               
            S_total_noisy_array = S_total_numpy + S_total_noise                                                                                                                                                                              

            A_total_noise = np.random.normal(0, 1, size = A_total_numpy.shape) 
            A_total_noise = A_total_noise * 0.1 * A_total_numpy
            A_total_noisy_array = A_total_numpy + A_total_noise

            B_total_noise = np.random.normal(0, 1, size = B_total_numpy.shape) 
            B_total_noise = B_total_noise * 0.1 * B_total_numpy
            B_total_noisy_array = B_total_numpy + B_total_noise

            P_total_noise = np.random.normal(0, 1, size = P_total_numpy.shape) 
            P_total_noise = P_total_noise * 0.1 * P_total_numpy
            P_total_noisy_array = P_total_numpy + P_total_noise

            jsa_noise = np.random.normal(0, 1, size = jsa_numpy.shape)        
            jsa_noise = jsa_noise * 0.02 * jsa_numpy                          
            jsa_noisy_array = jsa_numpy + jsa_noise
            
            jab_noise = np.random.normal(0, 1, size = jab_numpy.shape)        
            jab_noise = jab_noise * 0.02 * jab_numpy
            jab_noisy_array = jab_numpy + jab_noise
            
            jbp_noise = np.random.normal(0, 1, size = jbp_numpy.shape)       
            jbp_noise = jbp_noise * 0.02 * jbp_numpy
            jbp_noisy_array = jbp_numpy + jbp_noise
            
            jab_b_noise = np.random.normal(0, 1, size = jab_b_numpy.shape)    # Adding synthetic noise to our reversible fluxes
            jab_b_noise = jab_b_noise * 0.02 * jab_b_numpy
            jab_b_noisy_array = jab_b_numpy + jab_b_noise
            
            jbp_b_noise = np.random.normal(0, 1, size = jbp_b_numpy.shape)    
            jbp_b_noise = jbp_b_noise * 0.02 * jbp_b_numpy
            jbp_b_noisy_array = jbp_b_numpy + jbp_b_noise
            
            fs_noise = np.random.normal(0, 1, size = fs_numpy.shape)      
            fs_noise = fs_noise * 0.02 * fs_numpy                        
            fs_noisy_array = fs_numpy + fs_noise

            fa_noise = np.random.normal(0, 1, size = fa_numpy.shape)     
            fa_noise = fa_noise * 0.02 * fa_numpy
            fa_noisy_array = fa_numpy + fa_noise

            fb_noise = np.random.normal(0, 1, size = fb_numpy.shape)     
            fb_noise = fb_noise * 0.02 * fb_numpy
            fb_noisy_array = fb_numpy + fb_noise

            fp_noise = np.random.normal(0, 1, size = fp_numpy.shape)     
            fp_noise = fp_noise * 0.02 * fp_numpy
            fp_noisy_array = fp_numpy + fp_noise
            
            jsa_net_noisy_array = jsa_noisy_array                         # Calculating noisy net fluxes from the individual
            jab_net_noisy_array = jab_noisy_array - jab_b_noisy_array     # noisy forward and reverse fluxes
            jbp_net_noisy_array = jbp_noisy_array - jbp_noisy_array 
            
            results_dictionary = {'Times': times_numpy,                          
                                  'Total_S': S_total_noisy_array,                    
                                  'Total_A': A_total_noisy_array,
                                  'Total_B': B_total_noisy_array,
                                  'Total_P': P_total_noisy_array,
                                  'SA_Flux': jsa_noisy_array,
                                  'AB_Flux': jab_noisy_array,
                                  'BP_Flux': jbp_noisy_array,
                                  'AB_b_Flux': jab_b_noisy_array,
                                  'BP_b_Flux': jbp_b_noisy_array,
                                  'SA_Net_Flux': jsa_net_noisy_array,
                                  'AB_Net_Flux': jab_net_noisy_array,
                                  'BP_Net_Flux': jbp_net_noisy_array,
                                  'fs': fs_noisy_array,
                                  'fa': fa_noisy_array,
                                  'fb': fb_noisy_array,
                                  'fp': fp_noisy_array}
            
            results_df = pd.DataFrame(results_dictionary)
            results_df.to_csv('GroupX_FirstOrderData_Subsampled.csv')
            
            print('Exported Data!')

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions we need for our interface. 
#----------------------------------------------------------------------------------------------------------------

filename = ''

def initialize_elements(total_time_slider_value = 100, timestep_slider_value = 0.1,                   
                       switch_point_slider_value = 20, S_slider_value = 0, A_slider_value = 0,
                       B_slider_value=0, P_slider_value = 0, LS_slider_value = 100,
                       LA_slider_value = 0, LB_slider_value = 0, LP_slider_value = 0,
                       S_switch_slider_value = 0, LS_switch_slider_value = 0,
                       k_SA_slider_value = 0.1, k_AB_slider_value = 0.1, k_AB_b_slider_value = 0.01,
                       k_BP_slider_value = 0.1, k_BP_b_slider_value = 0.01, compare_drop_value = 'None', subsample_drop_value = False,
                       export_drop_value=False):

    dictionary_of_elements = {'total_time_slider': widgets.IntSlider(min= 1, max = 1000, step = 1, value = total_time_slider_value),
                              'timestep_slider': widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = timestep_slider_value),
                              'switch_point_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = switch_point_slider_value),
                              'S_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = S_slider_value),
                              'A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = A_slider_value),
                              'B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = B_slider_value),
                              'P_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = P_slider_value),
                              'LS_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LS_slider_value),
                              'LA_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LA_slider_value),
                              'LB_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LB_slider_value),
                              'LP_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LP_slider_value),
                              'S_switch_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = S_switch_slider_value),
                              'LS_switch_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LS_switch_slider_value),
                              'k_SA_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_SA_slider_value),
                              'k_AB_slider': widgets.FloatSlider(min= 0 , max = 1, step = 0.01, value = k_AB_slider_value),
                              'k_AB_b_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_AB_b_slider_value),
                              'k_BP_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_BP_slider_value),
                              'k_BP_b_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_BP_b_slider_value),
                              'compare_drop': widgets.Dropdown(options = [('None', 'None'),('None', 'None'),
                                                                        ('Just Concentration','Just Concentration'),
                                                                        ('ConcentrationAndFractionalLabeling','ConcentrationAndFractionalLabeling'),
                                                                        ('Everything','Everything')], value = compare_drop_value),
                              'subsample_drop': widgets.Dropdown(options = [('Yes', True),('No', False)], value = subsample_drop_value),
                              'export_drop': widgets.Dropdown(options = [('Yes', True),('No', False)], value = export_drop_value),
                              'button': widgets.Button(description = 'Run')}
    return dictionary_of_elements


def on_button_clicked(_):                          
    with out:
        clear_output()
        reversible_simulation_interactive()
        show_inline_matplotlib_plots()
        
def reversible_simulation_interactive():                             
    total_time = elements['total_time_slider'].value
    timestep = elements['timestep_slider'].value
    switch_point = elements['switch_point_slider'].value
    S = elements['S_slider'].value
    A = elements['A_slider'].value
    B = elements['B_slider'].value
    P = elements['P_slider'].value
    LS = elements['LS_slider'].value
    LA = elements['LA_slider'].value
    LB = elements['LB_slider'].value
    LP = elements['LP_slider'].value
    S_switch = elements['S_switch_slider'].value
    LS_switch = elements['LS_switch_slider'].value
    k_SA = elements['k_SA_slider'].value
    k_AB = elements['k_AB_slider'].value
    k_AB_b = elements['k_AB_b_slider'].value
    k_BP = elements['k_BP_slider'].value
    k_BP_b = elements['k_BP_b_slider'].value
    export = elements['export_drop'].value
    compare = elements['compare_drop'].value
    subset = elements['subsample_drop'].value

    reversible_kinetic_simulation(total_time, timestep, switch_point, S, A, B, P, LS, LA, LB,
        LP, S_switch, LS_switch, k_SA, k_AB, k_AB_b, k_BP, 
          k_BP_b,export,compare,filename,subset)

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements()                        

elements['button'].on_click(on_button_clicked)          
                                                        
out = widgets.Output()                                 

grid = widgets.GridspecLayout(8, 6)                             

grid[0,0] = widgets.HTML('<b>Total Time</b>') 
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Switch Point</b>')
grid[0,5] = elements['switch_point_slider']

grid[1,0] = widgets.HTML('<b>Pre-Switch Constant [S]</b>')
grid[1,1] = elements['S_slider']
grid[1,2] = widgets.HTML('<b>Initial [A]</b>')
grid[1,3] = elements['A_slider']
grid[1,4] = widgets.HTML('<b>Initial [B]</b>')
grid[1,5] = elements['B_slider']

grid[2,0] = widgets.HTML('<b>Initial [P]</b>')
grid[2,1] = elements['P_slider']
grid[2,2] = widgets.HTML('<b>Pre-Switch Constant [LS]</b>')
grid[2,3] = elements['LS_slider']
grid[2,4] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,5] = elements['LA_slider']

grid[3,0] = widgets.HTML('<b>Initial [LB]</b>')
grid[3,1] = elements['LB_slider']
grid[3,2] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,3] = elements['LP_slider']
grid[3,4] = widgets.HTML('<b>Post-Switch Constant [S]</b>')
grid[3,5] = elements['S_switch_slider']

grid[4,0] = widgets.HTML('<b>Post-Switch Constant [LS]</b>')
grid[4,1] = elements['LS_switch_slider']
grid[4,2] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[4,3] = elements['k_SA_slider']
grid[4,4] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,5] = elements['k_AB_slider']

grid[5,0] = widgets.HTML('<b>B -> P Rate Constant</b>')
grid[5,1] = elements['k_BP_slider']
grid[5,2] = widgets.HTML('<b>B -> A Rate Constant</b>')
grid[5,3] = elements['k_AB_b_slider']
grid[5,4] = widgets.HTML('<b>P -> B Rate Constant</b>')
grid[5,5] = elements['k_BP_b_slider']

grid[6,0] = widgets.HTML('<b>Export?</b>')
grid[6,1] = elements['export_drop']
grid[6,2] = widgets.HTML('<b>Compare?</b>')
grid[6,3] = elements['compare_drop']
grid[6,4] = widgets.HTML('<b>Subsample?</b>')
grid[6,5] = elements['subsample_drop']

grid[7,0] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Discuss the exercise results within your group. What difference does adding in reverse reactions make? Do you think this might make fitting parameters harder? Is it possible for our simulation results with reversible reactions to show behavior that you wouldn't be able to recreate with just first-order kinetics? 
</div>

# **(2.0)** Parameter fitting with reversibility

More parameters means more possibilities, and more possibilities means trickier parameter fitting, whether you're doing things by eye or using a computer to automate the process. An additional problem that crops up when you introduce more parameters is that you may potentially run into multiple sets of parameters that do an equally good job of fitting your data. If these parameter estimates are quite different from each other, that's a big problem, because you'll end up uncertain about what the **true** parameter values are. Both the amount and types of data you have determine the complexity of the model you can potentially justify. Let's get a feel for the increasing difficulty of fitting we incur as we add more parameters.

<div class="alert alert-block alert-info">

**Instructions:** Run the two code blocks below; the first defines the functions we'll need to execute the simulation and the second opens up an interactive widget for you to play around with it. Come up with some interesting looking plots by varying your parameters and then use the export function to export a subsampled dataset. Then, exchange your data with another group.
    
</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements(subsample_drop_value=True)      

elements['button'].on_click(on_button_clicked)          
                                                        
out = widgets.Output()                                  

grid = widgets.GridspecLayout(8, 6)                             

grid[0,0] = widgets.HTML('<b>Total Time</b>') 
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Switch Point</b>')
grid[0,5] = elements['switch_point_slider']

grid[1,0] = widgets.HTML('<b>Pre-Switch Constant[S]</b>')
grid[1,1] = elements['S_slider']
grid[1,2] = widgets.HTML('<b>Initial [A]</b>')
grid[1,3] = elements['A_slider']
grid[1,4] = widgets.HTML('<b>Initial [B]</b>')
grid[1,5] = elements['B_slider']

grid[2,0] = widgets.HTML('<b>Initial [P]</b>')
grid[2,1] = elements['P_slider']
grid[2,2] = widgets.HTML('<b>Pre-Switch Constant[LS]</b>')
grid[2,3] = elements['LS_slider']
grid[2,4] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,5] = elements['LA_slider']

grid[3,0] = widgets.HTML('<b>Initial [LB]</b>')
grid[3,1] = elements['LB_slider']
grid[3,2] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,3] = elements['LP_slider']
grid[3,4] = widgets.HTML('<b>Post-Switch Constant [S]</b>')
grid[3,5] = elements['S_switch_slider']

grid[4,0] = widgets.HTML('<b>Post-Switch Constant [LS]</b>')
grid[4,1] = elements['LS_switch_slider']
grid[4,2] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[4,3] = elements['k_SA_slider']
grid[4,4] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,5] = elements['k_AB_slider']

grid[5,0] = widgets.HTML('<b>B -> P Rate Constant</b>')
grid[5,1] = elements['k_BP_slider']
grid[5,2] = widgets.HTML('<b>B -> A Rate Constant</b>')
grid[5,3] = elements['k_AB_b_slider']
grid[5,4] = widgets.HTML('<b>P -> B Rate Constant</b>')
grid[5,5] = elements['k_BP_b_slider']

grid[6,0] = widgets.HTML('<b>Export?</b>')
grid[6,1] = elements['export_drop']
grid[6,2] = widgets.HTML('<b>Compare?</b>')
grid[6,3] = elements['compare_drop']
grid[6,4] = widgets.HTML('<b>Subsample?</b>')
grid[6,5] = elements['subsample_drop']

grid[7,0] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-info">

**Instructions:** Run the two code cells below and try to fit the dataset you were provided with.
    
</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# Change this filename to the name of the file you received from the other group.
#----------------------------------------------------------------------------------------------------------------

filename = 'Day02_GroupX_PartialData.csv'

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements(compare_drop_value='ConcentrationAndFractionalLabeling')   

elements['button'].on_click(on_button_clicked)                                                                  

out = widgets.Output()                                  

grid = widgets.GridspecLayout(8, 6)                             

grid[0,0] = widgets.HTML('<b>Total Time</b>') 
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Switch Point</b>')
grid[0,5] = elements['switch_point_slider']

grid[1,0] = widgets.HTML('<b>Pre-Switch Constant [S]</b>')
grid[1,1] = elements['S_slider']
grid[1,2] = widgets.HTML('<b>Initial [A]</b>')
grid[1,3] = elements['A_slider']
grid[1,4] = widgets.HTML('<b>Initial [B]</b>')
grid[1,5] = elements['B_slider']

grid[2,0] = widgets.HTML('<b>Initial [P]</b>')
grid[2,1] = elements['P_slider']
grid[2,2] = widgets.HTML('<b>Pre-Switch Constant [LS]</b>')
grid[2,3] = elements['LS_slider']
grid[2,4] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,5] = elements['LA_slider']

grid[3,0] = widgets.HTML('<b>Initial [LB]</b>')
grid[3,1] = elements['LB_slider']
grid[3,2] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,3] = elements['LP_slider']
grid[3,4] = widgets.HTML('<b>Post-Switch Constant [S]</b>')
grid[3,5] = elements['S_switch_slider']

grid[4,0] = widgets.HTML('<b>Post-Switch Constant [LS]</b>')
grid[4,1] = elements['LS_switch_slider']
grid[4,2] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[4,3] = elements['k_SA_slider']
grid[4,4] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,5] = elements['k_AB_slider']

grid[5,0] = widgets.HTML('<b>B -> P Rate Constant</b>')
grid[5,1] = elements['k_BP_slider']
grid[5,2] = widgets.HTML('<b>B -> A Rate Constant</b>')
grid[5,3] = elements['k_AB_b_slider']
grid[5,4] = widgets.HTML('<b>P -> B Rate Constant</b>')
grid[5,5] = elements['k_BP_b_slider']

grid[6,0] = widgets.HTML('<b>Export?</b>')
grid[6,1] = elements['export_drop']
grid[6,2] = widgets.HTML('<b>Compare?</b>')
grid[6,3] = elements['compare_drop']
grid[6,4] = widgets.HTML('<b>Subsample?</b>')
grid[6,5] = elements['subsample_drop']

grid[7,0] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Discuss the exercise results within your group. Was this harder than when you were fitting the irreversible first order models? Have you encountered this tradeoff between model complexity/realism and ease-of-fitting in your own work?
</div>

# **(3.0)** Metabolic Control Analysis

Let's say you're a bioengineer tasked with making yeast produce as much of a certain compound as possible in large bioreactors. You know the individual enzymes in the pathway that leads to your product of interest and you vaguely remember from your undergraduate biochemistry class that there's something called a "rate-limiting step" in any pathway. One of the reactions in your pathway is the slowest one and that one should be rate-limiting, right? And if that's the case, that should be our target. After all, increasing the expression of the enzymes that **aren't** rate limiting won't make any difference. So, you overexpress the gene(s) encoding the enzyme catalyzing the rate-limiting step, do a Western to confirm that the protein product is present, etc. But when you do the final test of whether your cultures are making way more of your product than before, the results are ... disappointing. 

So, What happened? 

## **(3.1)** Control over flux through a pathway is distributed across the enzymes in that pathway

Let's take a slightly altered version of the system we worked with last time, shown below:

![alt text](MCA_Network.png "Title")

We have enzymes catalyzing the reversible reactions between S and A, A, and B, B and P, and P and our final product. The flux we really care about is V4 - that's what makes the product we want. It turns out that for a pathway like the one above, control of the flux through the pathway is actually distributed between all of the enzymes catalyzing its reactions. Another way of putting this is that under a given set of conditions, varying the concentration of any of the enzymes will have some effect on the flux **Jpx**. Note that we've been ignoring enzyme concentrations up until now, but in the reversible first-order kinetics we're using, each of the first-order rate constants we were using before can be thought of as the product of a rate constant and an enzyme concentration. This means ...

$$ k_1 = k_1'[E]_{Jsa} $$
$$ k_{-1} = k_{-1}'[E]_{Jsa} $$
$$ k_2 = k_2'[E]_{Jab} $$
$$ k_{-2} = k_{-2}'[E]_{Jab} $$
$$ k_3 = k_3'[E]_{Jbp} $$
$$ k_{-3} = k_{-3}'[E]_{Jbp} $$
$$ k_4 = k_4'[E]_{Jpx} $$

By varying each of these concentrations up and down, we can observe how the flux **Jpx** responds. Let's focus on the enzyme catalyzing **Jab**. If we plot its concentration on the X axis and the flux through **Jpx** on the Y axis, we'll have a curve like the one below:

<p align="center">
<img src="concvsfluxExample.png" />
</p>


We can pick a particular concentration, ideally one that is physiologically realistic, and take the slope of the tangent line (or the first derivative) at that point along the line. This will represent the rate at which the flux **Jpx** is changing with enzyme concentration. But, this value will be a little hard to interpret since it's dependent on the specific units we use for enzyme concentration and flux, so we typically focus instead on fractional changes. Focusing again on the enzyme catalyzing **Jab**, we can calculate the flux control coefficient as:

$$ C_{[E]_{Jab}}^{Jpx} = \frac{\partial Jpx}{Jpx} / \frac{\partial [E]_{Jab}}{[E]_{Jab}} $$

This can be equivalently written as:

$$ C_{[E]_{Jab}}^{Jpx} = \frac{\partial \ln Jpx}{\partial \ln [E]_{Jab}} $$

If we do this for the four enzymes catalyzing the four reactions in our pathway, we'll have four terms representing the fraction of total control each enzyme exerts over the final flux **Jpx**. Since all of these terms represent the relative control of an enzyme, they should add up to 1, or 100% control:

$$ C_{[E]_{Jsa}}^{Jpx} + C_{[E]_{Jab}}^{Jpx} + C_{[E]_{Jbp}}^{Jpx} + C_{[E]_{Jpx}}^{Jpx} = 1 $$

## **(3.2)** Hands-on MCA Exercise

Let's try doing MCA ourselves. Each group will receive an Excel spreadsheet with a set of parameters to set. These will include enzyme concentrations at which we want to evaluate control coefficients.

<div class="alert alert-block alert-info">

**Instructions:** Run the two cells below. In the interactive cell, make sure you get your conditions set up correctly. Then, for each enzyme, vary its concentration up-and-down from its original concentration. You will get the flux through **Jpx** as a readout. Record all of this data in the Excel spreadsheet you were given, then plot out the data and find the tangent line of the *ln* Jpx ~ *ln* [E] plot. This will be your control coefficient. Once every group has their values, we'll all chat about the coefficients we found and how they may relate to the unique conditions we were looking at. 
    
</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell we are defining functions that we will need for MCA. 
#-----------------------------------------------------------------------------------------------------------------------

def evaluate_flux_wEnzyme(S, K, enzyme):                           # Defining evaluate_flux_wEnzyme(), which we'll use to 
                                                                   # calculate fluxes in our simulation. Note that we 
                                                                   # are now taking enzyme concentration into account

    '''
    DESCRIPTION
    
    Takes a rate constant k and a substrate concentration S 
    and returns a flux value.
    
    INPUTS
    
    k = a numeric first order rate constant
    
    S = a numeric concentration of a substrate
    
    enzyme = a numeric concentration of an enzyme
    
    OUTPUTS
    
    flux = flux, or reaction rate, given substrate 
    concentration, first order rate constant, and
    enzyme concentration
    '''

    flux = K * S * enzyme
    return flux

def evaluate_dLAdt_b(Jsa_b, Jab_b, f_a, f_b):                         # Since we've made the flux between S and A reversible, 
                                                                      # we add a function to calculate this value
    dLAdt = Jab_b * f_b - Jsa_b * f_a
    return dLAdt

def evaluate_dLPdt_f(Jbp, Jpx, f_b, f_p):                             # Redefining our functions for the change in [P] because
                                                                      # we have a new flux that interfaces with this metabolite

    dLPdt = Jbp * f_b - Jpx * f_p
    return dLPdt

def evaluate_dLPdt_b(Jbp_b, f_p):
    dLPdt = -Jbp_b * f_p
    return dLPdt


In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# For our MCA simulation, we are not going to switch substrate conditions mid-simulation - instead, we are going to let
# the system run to steady-state and then make measurements at that steady-state. We also add in modifiable parameters
# for enzyme concentrations so we can see how our final flux out of the network changes as a function of [enzyme] for the
# variosu reactions. Otherwise, the simulation code is quite similar. 
#-----------------------------------------------------------------------------------------------------------------------

def MCA(total_time = 10000, timestep = 1, S = 10, A = 0, B = 0, P = 0, LS = 0, LA = 0, LB = 0,
        LP = 0, K_SA = 0.1, K_SA_b = 0.1, K_AB = 0.1, K_AB_b = 0.01,
        K_BP = 0.1, K_BP_b = 0.01, K_PX = 0.1, enzyme_SA = 1, enzyme_AB = 1, enzyme_BP = 1, enzyme_PX = 1, mode = 'flux'): 
    
    '''
    DESCRIPTION
    
    A function that carries out a metabolic modeling simulation using reversible first-order kinetics.
    Samples flux values for a final pathway flux at steady-state and prints out the results in a plot.
    
    INPUTS
    
    total_time = A numeric value with arbitrary units that determines how long to run
    the simulation for.
    
    timestep = A numeric value representing how much to increment the time value each
    loop of the simulation.
    
    S = A numeric value representing the set concentration of unlabeled S 
    
    A = A numeric value representing the initial concentration of unlabeled A
    
    B = A numeric value representing the initial concentration of unlabeled B
    
    P = A numeric value representing the initial concentration of unlableed P
    
    LS = A numeric value representing the set concentration of labeled S
    
    LA = A numeric value representing the initial concentration of labeled A
    
    LB = A numeric value representing the initial concentration of labeled B
    
    LP = A numeric value representing the initial concentration of labeled P
    
    k_SA = A numeric value representing the first-order rate constant for the
    reaction S -> A
    
    k_SA_b = A numeric value representing the first-order rate constant for the
    reaction A -> S
    
    k_AB = A numeric value represnting the first-order rate constant for the 
    reaction A -> B
    
    k_AB_b = A numeric value representing the first-order rate constant for the 
    reaction B -> A
    
    k_BP = A numeric value representing the first-order rate constant for the 
    reaction B -> P
    
    k_BP_b = A numeric value representing the first-order rate constant for the
    reaction P -> B
    
    k_PX = A numeric value representing the first-order rate constant for the 
    reaction P -> X
    
    enzyme_SA = A numeric value representing the concentration of the enzyme
    catalyzing reaction S <=> A
    
    enzyme_AB = A numeric value representing the concentration of the enzyme
    catalyzing reaction A <=> B
    
    enzyme_BP = A numeric value representing the concentration of the enzyme
    catalyzing reaction B <=> P
    
    enzyme_PX = A numeric value representing the concentration of the enzyme
    catalyzing reaction P -> X
    
    mode = a string value that sets the function to return flux control coefficients
    'flux' or concentration control coefficients 'concentration'
    
    OUTPUTS
    
    None
    
    '''
        
    times = np.arange(0, total_time, timestep)                  
    
    S_total = S + LS                                             
    A_total = A + LA                                             
    B_total = B + LB
    P_total = P + LP
    
    S_list = [S]                                                                                    
    A_list = [A]                                                                               
    B_list = [B]                                   
    P_list = [P]                                    
                                                   
    LS_list = [LS]                                 
    LA_list = [LA]
    LB_list = [LB]
    LP_list = [LP]
        
    S_total_list = [S_total]                        
    A_total_list = [A_total]
    B_total_list = [B_total]
    P_total_list = [P_total]
    
    fs_list = [evaluate_fractional_labeling(LS, S_total)]               
    fa_list = [evaluate_fractional_labeling(LA, A_total)]              
    fb_list = [evaluate_fractional_labeling(LB, B_total)]               
    fp_list = [evaluate_fractional_labeling(LP, P_total)]
    
    jsa_list = [evaluate_flux_wEnzyme(S_total_list[-1], K_SA,enzyme_SA)]         
    jsa_b_list = [evaluate_flux_wEnzyme(A_total_list[-1], K_SA_b,enzyme_SA)]
    jab_list = [evaluate_flux_wEnzyme(A_total_list[-1], K_AB,enzyme_AB)]
    jab_b_list = [evaluate_flux_wEnzyme(B_total_list[-1], K_AB_b,enzyme_AB)]
    jbp_list = [evaluate_flux_wEnzyme(B_total_list[-1], K_BP,enzyme_BP)]
    jbp_b_list = [evaluate_flux_wEnzyme(P_total_list[-1], K_BP_b,enzyme_BP)]
    jpx_list = [evaluate_flux_wEnzyme(P_total_list[-1], K_PX,enzyme_PX)]
    
    for i in times:                                 
        
        current_S = S                               
        current_LS = LS                              
    
        current_Jsa = evaluate_flux_wEnzyme(S_total_list[-1], K_SA,enzyme_SA)          # Evaluating all our fluxes. Note that we are taking
        current_Jsa_b = evaluate_flux_wEnzyme(A_total_list[-1], K_SA_b,enzyme_SA)      # enzyme concentrations into account.
        current_Jab = evaluate_flux_wEnzyme(A_total_list[-1], K_AB,enzyme_AB)          
        current_Jab_b = evaluate_flux_wEnzyme(B_total_list[-1], K_AB_b,enzyme_AB)
        current_Jbp = evaluate_flux_wEnzyme(B_total_list[-1], K_BP,enzyme_BP)
        current_Jbp_b = evaluate_flux_wEnzyme(P_total_list[-1], K_BP_b,enzyme_BP)
        current_Jpx = evaluate_flux_wEnzyme(P_total_list[-1], K_PX,enzyme_PX)
        
        current_A = A_list[-1] + (evaluate_dLAdt_f(current_Jsa, current_Jab, (1 - fs_list[-1]), (1 - fa_list[-1])) * timestep)
        current_B = B_list[-1] + (evaluate_dLBdt_f(current_Jab, current_Jbp, (1 - fa_list[-1]), (1 - fb_list[-1])) * timestep)
        current_P = P_list[-1] + (evaluate_dLPdt_f(current_Jbp, current_Jpx, (1 - fb_list[-1]), (1 - fp_list[-1])) * timestep)
        current_LA = LA_list[-1] + (evaluate_dLAdt_f(current_Jsa, current_Jab, fs_list[-1], fa_list[-1]) * timestep)
        current_LB = LB_list[-1] + (evaluate_dLBdt_f(current_Jab, current_Jbp, fa_list[-1], fb_list[-1]) * timestep)
        current_LP = LP_list[-1] + (evaluate_dLPdt_f(current_Jbp, current_Jpx, fb_list[-1], fp_list[-1]) * timestep)
        
        current_S_total = current_S + current_LS
        current_A_total = current_A + current_LA
        current_B_total = current_B + current_LB
        current_P_total = current_P + current_LP        
        current_fs = evaluate_fractional_labeling(current_LS, current_S_total)
        current_fa = evaluate_fractional_labeling(current_LA, current_A_total)
        current_fb = evaluate_fractional_labeling(current_LB, current_B_total)
        current_fp = evaluate_fractional_labeling(current_LP, current_P_total)        
                
        current_A = current_A + (evaluate_dLAdt_b(current_Jsa_b, current_Jab_b, (1 - fa_list[-1]), (1 - fb_list[-1])) * timestep)
        current_B = current_B + (evaluate_dLBdt_b(current_Jab_b, current_Jbp_b, (1 - fb_list[-1]), (1 - fp_list[-1])) * timestep)
        current_P = current_P + (evaluate_dLPdt_b(current_Jbp_b, (1 - fp_list[-1])) * timestep)
        current_LA = current_LA + (evaluate_dLAdt_b(current_Jsa_b, current_Jab_b, fa_list[-1], fb_list[-1]) * timestep)
        current_LB = current_LB + (evaluate_dLBdt_b(current_Jab_b, current_Jbp_b, fb_list[-1], fp_list[-1]) * timestep)
        current_LP = current_LP + (evaluate_dLPdt_b(current_Jbp_b, fp_list[-1]) * timestep)
        
        current_S_total = current_S + current_LS
        current_A_total = current_A + current_LA
        current_B_total = current_B + current_LB
        current_P_total = current_P + current_LP 
        current_fs = evaluate_fractional_labeling(current_LS, current_S_total)
        current_fa = evaluate_fractional_labeling(current_LA, current_A_total)
        current_fb = evaluate_fractional_labeling(current_LB, current_B_total)
        current_fp = evaluate_fractional_labeling(current_LP, current_P_total)    
        
        LS_list.append(current_LS)
        LA_list.append(current_LA)
        LB_list.append(current_LB)
        LP_list.append(current_LP)

        S_list.append(current_S)
        A_list.append(current_A)
        B_list.append(current_B)
        P_list.append(current_P)

        S_total_list.append(current_S_total)
        A_total_list.append(current_A_total)
        B_total_list.append(current_B_total)
        P_total_list.append(current_P_total)

        fs_list.append(current_fs)
        fa_list.append(current_fa)
        fb_list.append(current_fb)
        fp_list.append(current_fp)
        
        jsa_list.append(current_Jsa)
        jsa_b_list.append(current_Jsa_b)
        jab_list.append(current_Jab)
        jab_b_list.append(current_Jab_b)
        jbp_list.append(current_Jbp)
        jbp_b_list.append(current_Jbp_b)
        jpx_list.append(current_Jpx)
    
    jsa_net = np.array(jsa_list) - np.array(jsa_b_list)              # We need net fluxes (specifically the net flux through Jpx) 
    jab_net = np.array(jab_list) - np.array(jab_b_list)              # to generate the ln flux vs. ln concentration graph we need
    jbp_net = np.array(jbp_list) - np.array(jbp_b_list)              
    jpx_net = np.array(jpx_list)                                                               

    
    times_including_initial = np.insert(times, 0, times[0] - timestep) 

    fig, ax = plt.subplots(1, 1, figsize = (12, 5))                

    # Plotting net fluxes
    if mode == 'flux':
        
        ax.plot(times_including_initial, jsa_net, label = 'S -> A; {0}'.format(enzyme_SA))
        ax.plot(times_including_initial, jab_net, label = 'A -> B; {0}'.format(enzyme_AB))
        ax.plot(times_including_initial, jbp_net, label = 'B -> P; {0}'.format(enzyme_BP))  
        ax.plot(times_including_initial, jpx_net, label = 'P -> X; {0}'.format(enzyme_PX))
        ax.legend()
        ax.set_title('Jpx Flux: {0}'.format(jpx_list[-1], fontweight = 'bold'))            # We use the title to convey the steady-state Jpx flux value 
        ax.set_xlabel('Time', fontweight='bold')
        ax.set_ylabel('Flux', fontweight='bold')
        
    if mode == 'concentration':
        ax.plot(times_including_initial, S_total_list, label = 'S; {0}'.format(S_total_list[-1]))
        ax.plot(times_including_initial, A_total_list, label = 'A; {0}'.format(A_total_list[-1]))
        ax.plot(times_including_initial, B_total_list, label = 'B; {0}'.format(B_total_list[-1]))  
        ax.plot(times_including_initial, P_total_list, label = 'P; {0}'.format(P_total_list[-1]))
        ax.legend()
        ax.set_title('[P]: {0}'.format(P_total_list[-1], fontweight = 'bold'))
        ax.set_xlabel('Time',fontweight = 'bold')
        ax.set_ylabel('Flux',fontweight = 'bold')

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions we need for our interface. 
#----------------------------------------------------------------------------------------------------------------

filename = ''

def initialize_elements_MCA(total_time_slider_value = 10000, timestep_slider_value = 1,                   
                       S_slider_value = 10, A_slider_value = 0, B_slider_value = 0, P_slider_value = 0, LS_slider_value = 0,
                       LA_slider_value = 0, LB_slider_value = 0, LP_slider_value = 0,
                       k_SA_slider_value = 0.1, k_SA_b_slider_value = 0.09, k_AB_slider_value = 0.1,
                       k_AB_b_slider_value = 0.01, k_BP_slider_value = 0.1, k_BP_b_slider_value = 0.01, k_PX_slider_value = 0.01,
                       enzyme_SA_slider_value = 0.3, enzyme_AB_slider_value = 0.1, enzyme_BP_slider_value = 0.1,
                       enzyme_PX_slider_value = 0.1, mode_drop_value = 'flux'):

    dictionary_of_elements = {'total_time_slider': widgets.IntSlider(min = 1, max = 10000, step = 1, value = total_time_slider_value),
                              'timestep_slider': widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = timestep_slider_value),
                              'S_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = S_slider_value),
                              'A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = A_slider_value),
                              'B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = B_slider_value),
                              'P_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = P_slider_value),
                              'LS_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LS_slider_value),
                              'LA_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LA_slider_value),
                              'LB_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LB_slider_value),
                              'LP_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LP_slider_value),
                              'k_SA_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_SA_slider_value),
                              'k_SA_b_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_SA_b_slider_value),
                              'k_AB_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_AB_slider_value),
                              'k_AB_b_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_AB_b_slider_value),
                              'k_BP_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_BP_slider_value),
                              'k_BP_b_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_BP_b_slider_value),
                              'k_PX_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_PX_slider_value),
                              'enzyme_SA_slider':widgets.FloatSlider(min = 0.01, max = 1.0, step = 0.01, value = enzyme_SA_slider_value),
                              'enzyme_AB_slider':widgets.FloatSlider(min = 0.01, max = 1.0, step = 0.01, value = enzyme_AB_slider_value),
                              'enzyme_BP_slider':widgets.FloatSlider(min = 0.01, max = 1.0, step = 0.01, value = enzyme_BP_slider_value),
                              'enzyme_PX_slider':widgets.FloatSlider(min = 0.01, max = 1.0, step = 0.01, value = enzyme_PX_slider_value),
                              'mode_drop': widgets.Dropdown(options = [('flux', 'flux'), ('concentration', 'concentration')],value = mode_drop_value),
                              'button': widgets.Button(description = 'Run')}

    return dictionary_of_elements

def on_button_clicked(_):                         
    with out:
        clear_output()
        MCA_interactive()
        show_inline_matplotlib_plots()
        
def MCA_interactive():                             
    total_time = elements['total_time_slider'].value
    timestep = elements['timestep_slider'].value
    S = elements['S_slider'].value
    A = elements['A_slider'].value
    B = elements['B_slider'].value
    P = elements['P_slider'].value
    LS = elements['LS_slider'].value
    LA = elements['LA_slider'].value
    LB = elements['LB_slider'].value
    LP = elements['LP_slider'].value
    k_SA = elements['k_SA_slider'].value
    k_SA_b = elements['k_SA_b_slider'].value
    k_AB = elements['k_AB_slider'].value
    k_AB_b = elements['k_AB_b_slider'].value
    k_BP = elements['k_BP_slider'].value
    k_BP_b = elements['k_BP_b_slider'].value
    k_PX = elements['k_PX_slider'].value
    enzyme_SA = elements['enzyme_SA_slider'].value
    enzyme_AB = elements['enzyme_AB_slider'].value
    enzyme_BP = elements['enzyme_BP_slider'].value
    enzyme_PX = elements['enzyme_PX_slider'].value
    mode = elements['mode_drop'].value

    MCA(total_time, timestep, S, A, B, P, LS, LA, LB,
        LP, k_SA, k_SA_b, k_AB,k_AB_b, k_BP, k_BP_b, k_PX,
          enzyme_SA, enzyme_AB, enzyme_BP, enzyme_PX, mode)
    

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements_MCA()                       

elements['button'].on_click(on_button_clicked)             
                                                           

out = widgets.Output()                                     

grid = widgets.GridspecLayout(8, 6)                                

grid[0,0] = widgets.HTML('<b>Total Time</b>') 
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Constant [S]</b>')
grid[0,5] = elements['S_slider']


grid[1,0] = widgets.HTML('<b>Initial [A]</b>')
grid[1,1] = elements['A_slider']
grid[1,2] = widgets.HTML('<b>Initial [B]</b>')
grid[1,3] = elements['B_slider']
grid[1,4] = widgets.HTML('<b>Initial [P]</b>')
grid[1,5] = elements['P_slider']

grid[2,0] = widgets.HTML('<b>Constant [LS]</b>')
grid[2,1] = elements['LS_slider']
grid[2,2] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,3] = elements['LA_slider']
grid[2,4] = widgets.HTML('<b>Initial [LB]</b>')
grid[2,5] = elements['LB_slider']

grid[3,0] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,1] = elements['LP_slider']
grid[3,2] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[3,3] = elements['k_SA_slider']
grid[3,4] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[3,5] = elements['k_SA_b_slider']

grid[4,0] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,1] = elements['k_AB_slider']
grid[4,2] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,3] = elements['k_AB_b_slider']
grid[4,4] = widgets.HTML('<b>B -> P Rate Constant</b>')
grid[4,5] = elements['k_BP_slider']

grid[5,0] = widgets.HTML('<b>P -> B Rate Constant</b>')
grid[5,1] = elements['k_BP_b_slider']
grid[5,2] = widgets.HTML('<b>P -> X Rate Constant</b>')
grid[5,3] = elements['k_PX_slider']
grid[5,4] = widgets.HTML('<b>[SA Enzyme]</b>')
grid[5,5] = elements['enzyme_SA_slider']

grid[6,0] = widgets.HTML('<b>[AB Enzyme]</b>')
grid[6,1] = elements['enzyme_AB_slider']
grid[6,2] = widgets.HTML('<b>[BP Enzyme]</b>')
grid[6,3] = elements['enzyme_BP_slider']
grid[6,4] = widgets.HTML('<b>[PX Enzyme]</b>')
grid[6,5] = elements['enzyme_PX_slider']

grid[7,0] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Once you've filled out your spreadsheet with the values you found, plotted everything out, and gotten estimates for the control coefficients of all four enzymes in the pathway, share your findings with the rest of the class. While you're waiting for other groups to finish up, discuss amongst your group members whether your results make sense. 
</div>

<div class="alert alert-block alert-warning"> 
    <b>Break Time</b> Once all groups have finished discussing their findings, we'll take a fifteen minute break.

## **(3.3)** Concentration Control Coefficients

When thinking about these metabolic networks, concentrations and fluxes are clearly related, but nevertheless distinct concepts and quantities. Just as we can define flux control coefficients for each enzyme showing how they impact the flux through the pathway **Jbp**, we can also define concentration control coefficients for each enzyme that show how they impact the steady-state concentration of a metabolite in our network. If we focus on metabolite **P**, we can define the concentration control coefficient for the enzyme catalyzing **Jab** ...

$$ C_{[E]_{Jab}}^{[P]} = \frac{\partial \ln [P]}{\partial \ln E_{Jab}} $$

Interestingly, the concentration control cofficients also sum up to a specific value:

$$ C_{[E]_{Jsa}}^{[P]} + C_{[E]_{Jab}}^{[P]} + C_{[E]_{Jbp}}^{[P]} + C_{[E]_{Jpx}}^{[P]} = 0 $$

It seems, intuitively, like an enzyme with a high/strong flux control coefficient for flux **Jpx** should have a high flux control cofficient for [P]. After all, what better way to push flux through reaction **Jpx** than to raise the concentration of metabolite **P**. Let's test this idea by repeating the above flux control coefficient exercise, but for the concentration control coefficient.

<div class="alert alert-block alert-info">

**Instructions:** Run the two cells below. In the interactive cell, make sure you get your conditions set up correctly. Then, for each enzyme, vary its concentration up-and-down from its original concentration. You will get the concentration of **P** as a readout. Record all of this data in the Excel spreadsheet you were given, then plot out the data and find the tangent line of the *ln* [P] ~ *ln* [E] plot. This will be your concentration control coefficient. Once every group has their values, we'll all chat about the coefficients we found and how they may relate to the unique conditions we were looking at. 
    
</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements_MCA(mode_drop_value='concentration')                       

elements['button'].on_click(on_button_clicked)             
                                                           

out = widgets.Output()                                    

grid = widgets.GridspecLayout(8, 6)                                

grid[0,0] = widgets.HTML('<b>Total Time</b>') 
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Constant [S]</b>')
grid[0,5] = elements['S_slider']


grid[1,0] = widgets.HTML('<b>Initial [A]</b>')
grid[1,1] = elements['A_slider']
grid[1,2] = widgets.HTML('<b>Initial [B]</b>')
grid[1,3] = elements['B_slider']
grid[1,4] = widgets.HTML('<b>Initial [P]</b>')
grid[1,5] = elements['P_slider']

grid[2,0] = widgets.HTML('<b>Constant [LS]</b>')
grid[2,1] = elements['LS_slider']
grid[2,2] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,3] = elements['LA_slider']
grid[2,4] = widgets.HTML('<b>Initial [LB]</b>')
grid[2,5] = elements['LB_slider']

grid[3,0] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,1] = elements['LP_slider']
grid[3,2] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[3,3] = elements['k_SA_slider']
grid[3,4] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[3,5] = elements['k_SA_b_slider']

grid[4,0] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,1] = elements['k_AB_slider']
grid[4,2] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,3] = elements['k_AB_b_slider']
grid[4,4] = widgets.HTML('<b>B -> P Rate Constant</b>')
grid[4,5] = elements['k_BP_slider']

grid[5,0] = widgets.HTML('<b>P -> B Rate Constant</b>')
grid[5,1] = elements['k_BP_b_slider']
grid[5,2] = widgets.HTML('<b>P -> X Rate Constant</b>')
grid[5,3] = elements['k_PX_slider']
grid[5,4] = widgets.HTML('<b>[SA Enzyme]</b>')
grid[5,5] = elements['enzyme_SA_slider']

grid[6,0] = widgets.HTML('<b>[AB Enzyme]</b>')
grid[6,1] = elements['enzyme_AB_slider']
grid[6,2] = widgets.HTML('<b>[BP Enzyme]</b>')
grid[6,3] = elements['enzyme_BP_slider']
grid[6,4] = widgets.HTML('<b>[PX Enzyme]</b>')
grid[6,5] = elements['enzyme_PX_slider']

grid[7,0] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Once you've filled out your spreadsheet with the values you found, plotted everything out, and gotten estimates for the concentration control coefficients of all four enzymes in the pathway, share your findings with the rest of the class. While you're waiting for other groups to finish up, discuss amongst your group members whether your results make sense. How do these results compare with the flux control coefficients?
</div>

## **(3.4)** Elasticity

Elasticity refers to the sensitivity of the flux through a particular enzymatic reaction to the concentration of a substrate or a product. That is, at a particular set of conditions, how much does changing the concentration of a substrate or product change the flux through a reaction? This could be useful to know if, for example, you had enzymes you could heterologously express in an organism to introduce an influx of a metabolite somewhere along a biosynthetic pathway of interest. If your goal is to increase a downstream flux but the metabolite whose steady state concentration you raised has very little effect on the next reaction in the pathway, that may not be an effective strategy ...

Elasticity coefficients are calculated similarly to the other coefficients we've encountered. Considering the reaction **A &#8594; B** and the metabolite **A**:

$$ \epsilon_{[A]}^{V_{ab}} = \frac{\partial \ln V_{ab}}{\partial \ln [A]} $$

Note that we are considering the reaction **A &#8594; B** in isolation, not in the context of the greater network, so we refer to its velocity and not its flux.

Now, let's give this a try ourselves. 

<div class="alert alert-block alert-info">

**Instructions:** In the cell above that we used to calculate concentration control coefficients, reenter your group's starting conditions and record the steady-state concentrations of metabolites S, A, B, and P. Once you've done that, run the cells below. You will test the sensitivity of each enzyme to both its substrate and product (except for the enzyme catalyzing Jpx, which you'll just test for sensitivity to [P]). Record your data for each enzyme and use it to calculate elasticities for substrate and product.
    
</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions that output reaction velocities given enzyme concentrations. 
#----------------------------------------------------------------------------------------------------------------

def evaluate_net_Jsa(S, A, k_SA, k_SA_b, enzyme_SA):                         # We provide the concentrations of the metabolites
    Jsa = k_SA * S * enzyme_SA                                               # involved in the reaction along with forward and
    Jsa_b = k_SA_b * A*enzyme_SA                                             # reverse rate constants and the enzyme concentration.
    Jsa_net = Jsa - Jsa_b                                                    # Using this information, we calculate the forward, reverse
                                                                             # and net reaction rates and print all of these out.
    print('[S] is:', S)
    print('[A] is:', A)
    print('The rate of the Jsa forward reaction is:', np.round(Jsa, 4))
    print('The rate of the Jsa reverse reaction is:', np.round(Jsa_b, 4))
    print('The rate of the Jsa net reaction is:', np.round(Jsa_net, 4))
    
def evaluate_net_Jab(A, B, k_AB, k_AB_b, enzyme_AB):                        # Same as above for Jab
    Jab = k_AB * A * enzyme_AB
    Jab_b = k_AB_b * B * enzyme_AB
    Jab_net = Jab - Jab_b
    
    print('[A] is:', A)
    print('[B] is:', B)
    print('The rate of the Jab forward reaction is:', np.round(Jab, 4))
    print('The rate of the Jab reverse reaction is:', np.round(Jab_b, 4))
    print('The rate of the Jab net reaction is:', np.round(Jab_net, 4))

def evaluate_net_Jbp(B, P, k_BP, k_BP_b, enzyme_BP):                      # Same as above for Jbp
    Jbp = k_BP * B * enzyme_BP
    Jbp_b = k_BP_b * P * enzyme_BP
    Jbp_net = Jbp - Jbp_b
    
    print('[B] is:', B)
    print('[P] is:', P)
    print('The rate of the Jbp forward reaction is:', np.round(Jbp, 4))
    print('The rate of the Jbp reverse reaction is:', np.round(Jbp_b, 4))
    print('The rate of the Jbp net reaction is:', np.round(Jbp_net, 4))
    
def evaluate_net_Jpx(P, k_PX, enzyme_PX):                                # Same as above for Jpx, though here we're just concerned
    Jpx = k_PX * P * enzyme_PX                                           # with the forward rate since this reaction is irreversible
    Jpx_net = Jpx 
    
    print('[P] is:', P)
    print('The rate of the Jbp net reaction is:', np.round(Jpx_net, 4))
        
def Jsa_interactive():                                                  # Below, we define the functions we need for our interactive
    S = S_slider.value                                                  # elements.
    A = A_slider.value
    k_SA = k_SA_slider.value
    k_SA_b = k_SA_b_slider.value
    enzyme = enzyme_slider.value
    
    evaluate_net_Jsa(S, A, k_SA, k_SA_b, enzyme)
    
def Jab_interactive():                                     
    A = A_slider.value
    B = B_slider.value
    k_AB = k_AB_slider.value
    k_AB_b = k_AB_b_slider.value
    enzyme = enzyme_slider.value
    
    evaluate_net_Jab(A, B, k_AB, k_AB_b, enzyme)
    
def Jbp_interactive():                                     
    B = B_slider.value
    P = P_slider.value
    k_BP = k_BP_slider.value
    k_BP_b = k_BP_slider.value
    enzyme = enzyme_slider.value
    
    evaluate_net_Jbp(B, P, k_BP, k_BP_b, enzyme)
    
def Jpx_interactive():                                     
    P = P_slider.value
    k_PX = k_PX_slider.value
    enzyme = enzyme_slider.value
    
    evaluate_net_Jpx(P, k_PX, enzyme)
    
def on_button_clicked_Jsa(_):                           
    with out:
        clear_output()
        Jsa_interactive()
        show_inline_matplotlib_plots()
        
def on_button_clicked_Jab(_):                           
    with out:
        clear_output()
        Jab_interactive()
        show_inline_matplotlib_plots()

def on_button_clicked_Jbp(_):                           
    with out:
        clear_output()
        Jbp_interactive()
        show_inline_matplotlib_plots()
        
def on_button_clicked_Jpx(_):                           
    with out:
        clear_output()
        Jpx_interactive()
        show_inline_matplotlib_plots()

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface to calculate elasticities for Jsa
#----------------------------------------------------------------------------------------------------------------

S_slider = widgets.IntSlider(min = 1, max = 100, step = 1, value = 5)
A_slider = widgets.IntSlider(min = 1, max = 100, step = 1, value = 5)
k_SA_slider = widgets.FloatSlider(min = 0.1, max = 10, step = 0.1, value = 1.0)
k_SA_b_slider = widgets.FloatSlider(min=0.1, max = 10, step = 0.1, value = 0.1)
enzyme_slider = widgets.FloatSlider(min=0.1, max = 10, step = 0.1, value = 0.1)
button = widgets.Button(description = 'Run')

button.on_click(on_button_clicked_Jsa)       
                                                       
out = widgets.Output()                                 

grid = widgets.GridspecLayout(2, 6)

grid[0,0] = widgets.HTML('<b>[S]</b>')
grid[0,1] = S_slider
grid[0,2] = widgets.HTML('<b>[A]</b>')
grid[0,3] = A_slider
grid[0,4] = widgets.HTML('<b>Ksa</b>')
grid[0,5] = k_SA_slider

grid[1,0] = widgets.HTML('<b>Ksa_b</b>')
grid[1,1] = k_SA_b_slider
grid[1,2] = widgets.HTML('<b>Enzyme conc.</b>')
grid[1,3] = enzyme_slider
grid[1,4] = button

display(grid,out)

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface to calculate elasticities for Jab
#----------------------------------------------------------------------------------------------------------------

A_slider = widgets.IntSlider(min = 1, max = 100, step = 1, value = 5)
B_slider = widgets.IntSlider(min = 1, max = 100, step = 1, value = 5)
k_AB_slider = widgets.FloatSlider(min = 0.1, max = 10, step = 0.1, value = 1.0)
k_AB_b_slider = widgets.FloatSlider(min = 0.1, max = 10, step = 0.1, value = 0.1)
enzyme_slider = widgets.FloatSlider(min = 0.1, max = 10, step = 0.1, value = 0.1)
button = widgets.Button(description='Run')

button.on_click(on_button_clicked_Jab)        
                                                       
out = widgets.Output()                                 

grid = widgets.GridspecLayout(2, 6)

grid[0,0] = widgets.HTML('<b>[A]</b>')
grid[0,1] = A_slider
grid[0,2] = widgets.HTML('<b>[B]</b>')
grid[0,3] = B_slider
grid[0,4] = widgets.HTML('<b>Kab</b>')
grid[0,5] = k_AB_slider

grid[1,0] = widgets.HTML('<b>Kab_b</b>')
grid[1,1] = k_AB_b_slider
grid[1,2] = widgets.HTML('<b>Enzyme conc.</b>')
grid[1,3] = enzyme_slider
grid[1,4] = button

display(grid,out)

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface to calculate elasticities for Jbp
#----------------------------------------------------------------------------------------------------------------

B_slider = widgets.IntSlider(min = 1, max = 100, step = 1, value = 5)
P_slider = widgets.IntSlider(min = 1, max = 100, step = 1, value = 5)
k_BP_slider = widgets.FloatSlider(min = 0.1, max = 10, step = 0.1, value = 1.0)
k_BP_b_slider = widgets.FloatSlider(min = 0.1, max = 10, step = 0.1, value = 0.1)
enzyme_slider = widgets.FloatSlider(min = 0.1, max = 10, step = 0.1, value = 0.1)
button = widgets.Button(description = 'Run')

button.on_click(on_button_clicked_Jbp)        
                                                       
out = widgets.Output()                                 

grid = widgets.GridspecLayout(2, 6)

grid[0,0] = widgets.HTML('<b>[B]</b>')
grid[0,1] = B_slider
grid[0,2] = widgets.HTML('<b>[P]</b>')
grid[0,3] = P_slider
grid[0,4] = widgets.HTML('<b>Kbp</b>')
grid[0,5] = k_BP_slider

grid[1,0] = widgets.HTML('<b>Kbp_b</b>')
grid[1,1] = k_BP_b_slider
grid[1,2] = widgets.HTML('<b>Enzyme conc.</b>')
grid[1,3] = enzyme_slider
grid[1,4] = button

display(grid,out)

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface to calculate elasticities for Jpx
#----------------------------------------------------------------------------------------------------------------

P_slider = widgets.IntSlider(min = 1, max = 100, step = 1, value = 5)
k_PX_slider = widgets.FloatSlider(min = 0.1, max = 10, step = 0.1, value = 1.0)
enzyme_slider = widgets.FloatSlider(min = 0.1, max = 10, step = 0.1, value = 0.1)
button = widgets.Button(description='Run')

button.on_click(on_button_clicked_Jpx)        
                                                       
out = widgets.Output()                                 

grid = widgets.GridspecLayout(2, 6)

grid[0,0] = widgets.HTML('<b>[P]</b>')
grid[0,1] = P_slider
grid[0,2] = widgets.HTML('<b>Kpx</b>')
grid[0,3] = k_PX_slider
grid[0,4] = widgets.HTML('<b>Enzyme conc.</b>')
grid[0,5] = enzyme_slider

grid[1,0] = button

display(grid,out)

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Discuss your results with the rest of your group. Do they make sense? Do they cohere with what you found in the other exercises? Report your results to the rest of the class and we'll have a whole class discussion.
</div>

# **(4.0)** Experimental Limitations in MCA

When working *in silico*, we can generate large amounts of synthetic data with extremely finely spaced timepoints or differences in concentration. However, empirical MCA studies involve calculating control coefficients from systems where enzyme activities (i.e. the x-axis on the *ln* Jpx and *ln* [E] plot) vary in at least 2-fold increments. This can lead to distortion of control coefficients. 

To see the effect this can have, we can compare the results we get when using realistic enzyme concentrations and a limited number of datapoints with the output of a function that calculates flux control coefficients from extremely finely spaced *in silico* datapoints.

<div class="alert alert-block alert-info">

**Instructions:** Run the cell below and perform MCA using the "experimental conditions" dataset from the Excel spreadsheet provided. Record the flux control coefficients for all enzymes in the pathway.
    
</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements_MCA()                       

elements['button'].on_click(on_button_clicked)             
                                                           

out = widgets.Output()                                     

grid = widgets.GridspecLayout(8, 6)                                

grid[0,0] = widgets.HTML('<b>Total Time</b>') 
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Constant [S]</b>')
grid[0,5] = elements['S_slider']


grid[1,0] = widgets.HTML('<b>Initial [A]</b>')
grid[1,1] = elements['A_slider']
grid[1,2] = widgets.HTML('<b>Initial [B]</b>')
grid[1,3] = elements['B_slider']
grid[1,4] = widgets.HTML('<b>Initial [P]</b>')
grid[1,5] = elements['P_slider']

grid[2,0] = widgets.HTML('<b>Constant [LS]</b>')
grid[2,1] = elements['LS_slider']
grid[2,2] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,3] = elements['LA_slider']
grid[2,4] = widgets.HTML('<b>Initial [LB]</b>')
grid[2,5] = elements['LB_slider']

grid[3,0] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,1] = elements['LP_slider']
grid[3,2] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[3,3] = elements['k_SA_slider']
grid[3,4] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[3,5] = elements['k_SA_b_slider']

grid[4,0] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,1] = elements['k_AB_slider']
grid[4,2] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,3] = elements['k_AB_b_slider']
grid[4,4] = widgets.HTML('<b>B -> P Rate Constant</b>')
grid[4,5] = elements['k_BP_slider']

grid[5,0] = widgets.HTML('<b>P -> B Rate Constant</b>')
grid[5,1] = elements['k_BP_b_slider']
grid[5,2] = widgets.HTML('<b>P -> X Rate Constant</b>')
grid[5,3] = elements['k_PX_slider']
grid[5,4] = widgets.HTML('<b>[SA Enzyme]</b>')
grid[5,5] = elements['enzyme_SA_slider']

grid[6,0] = widgets.HTML('<b>[AB Enzyme]</b>')
grid[6,1] = elements['enzyme_AB_slider']
grid[6,2] = widgets.HTML('<b>[BP Enzyme]</b>')
grid[6,3] = elements['enzyme_BP_slider']
grid[6,4] = widgets.HTML('<b>[PX Enzyme]</b>')
grid[6,5] = elements['enzyme_PX_slider']

grid[7,0] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-info">

**Instructions:** Now, run the cells below and plug in the same "experimental conditions" parameter values into the automated MCA function. Record your results.
    
</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# This code cell defines the functinos we need for our automated MCA pipeline.
#-----------------------------------------------------------------------------------------------------------------------

def evaluate_Jsa(S, K_SA, enzyme_SA):
    Jsa = K_SA * enzyme_SA * S
    return Jsa

def evaluate_Jsa_b(A, K_SA_b, enzyme_SA):
    Jsa = K_SA_b * enzyme_SA * A
    return Jsa

def evaluate_Jab(A, K_AB, enzyme_AB):
    Jab = K_AB * enzyme_AB * A
    return Jab

def evaluate_Jab_b(B, K_AB_b, enzyme_AB):
    Jab = K_AB_b * enzyme_AB * B
    return Jab

def evaluate_Jbp(B, K_BP, enzyme_BP):
    Jbp = K_BP * enzyme_BP * B
    return Jbp

def evaluate_Jbp_b(P, K_BP_b, enzyme_BP):
    Jbp = K_BP_b * enzyme_BP * P
    return Jbp

def evaluate_Jpx(P, K_PX, enzyme_PX):
    Jpx = K_PX * enzyme_PX * P 
    return Jpx

def evaluate_dLAdt(Jsa, Jsa_b, Jab, Jab_b, f_s, f_a, f_b):
    if (Jsa - Jsa_b) >= 0 and (Jab - Jab_b) >= 0:
        dLAdt = f_s * (Jsa - Jsa_b) - f_a * (Jab - Jab_b)
    elif (Jsa - Jsa_b) < 0 and (Jab - Jab_b) >= 0:
        dLAdt = f_a * (Jsa - Jsa_b) - f_a * (Jab - Jab_b)
    elif (Jsa - Jsa_b) >= 0 and (Jab - Jab_b) < 0:
        dLAdt = f_s * (Jsa - Jsa_b) - f_b * (Jab - Jab_b)
    elif (Jsa - Jsa_b) < 0 and (Jab - Jab_b) < 0:
        dLAdt = f_a * (Jsa - Jsa_b) - f_b * (Jab-Jab_b)
    return dLAdt

def evaluate_dLBdt(Jab, Jab_b, Jbp, Jbp_b, f_a, f_b, f_p):
    if (Jab - Jab_b) >= 0 and (Jbp - Jbp_b) >= 0:
        dLBdt = f_a * (Jab - Jab_b) - f_b * (Jbp - Jbp_b)
    elif (Jab - Jab_b) < 0 and (Jbp - Jbp_b) >= 0:
        dLBdt = f_a * (Jab - Jab_b) - f_b * (Jbp - Jbp_b)
    elif (Jab - Jab_b) >= 0 and (Jbp - Jbp_b) < 0:
        dLBdt = f_a * (Jab - Jab_b) - f_b * (Jbp - Jbp_b)
    elif (Jab - Jab_b) < 0 and (Jbp - Jbp_b) < 0:
        dLBdt = f_a * (Jab - Jab_b) - f_b * (Jbp - Jbp_b)
    return dLBdt

def evaluate_dLPdt(Jbp, Jbp_b, Jpx, f_b, f_p):
    if (Jbp - Jbp_b) >= 0:    
        dLPdt = (Jbp - Jbp_b) * f_b - Jpx * f_p
    elif (Jbp - Jbp_b) < 0:
        dLPdt = (Jbp - Jbp_b) * f_p - Jpx * f_p
    return dLPdt


def auto_MCA(total_time = 1000, timestep = 1, S = 0, A = 0, B = 0, P = 0, LS = 100, LA = 0, LB = 0,
        LP = 0, K_SA = 0.1, K_SA_b= 0.01, K_AB = 0.1, K_AB_b = 0.01,
        K_BP = 0.1, K_BP_b = 0.01, K_PX = 0.1, enzyme_SA = 0.9, enzyme_AB = 0.1, enzyme_BP = 0.1, enzyme_PX = 0.1): 
    
    enzymes = ['sa', 'ab', 'bp', 'px']
    concentrations = np.arange(0.01, 1, 0.01)
    fluxes_to_plot = []
    dydx_to_plot = []
    for enzyme in enzymes:
        jbp_values = []
        for concentration in concentrations:

            times = np.arange(0, total_time, timestep)

            S_total = S + LS
            A_total = A + LA
            B_total = B + LB
            P_total = P + LP

            S_list = [S]
            A_list = [A]
            B_list = [B]
            P_list = [P]

            LS_list = [LS]
            LA_list = [LA]
            LB_list = [LB]
            LP_list = [LP]


            S_total_list = [S_total]
            A_total_list = [A_total]
            B_total_list = [B_total]
            P_total_list = [P_total]

            fs_list = [evaluate_fractional_labeling(LS, S_total)]
            fa_list = [evaluate_fractional_labeling(LA, A_total)]
            fb_list = [evaluate_fractional_labeling(LB, B_total)]
            fp_list = [evaluate_fractional_labeling(LP, P_total)]

            #Lists for fluxes
            if enzyme == 'sa':
                jsa_list = [evaluate_Jsa(S_total_list[-1], K_SA, concentration)]
                jsa_b_list = [evaluate_Jsa_b(A_total_list[-1], K_SA_b, concentration)]
                jab_list = [evaluate_Jab(A_total_list[-1], K_AB,enzyme_AB)]
                jab_b_list = [evaluate_Jab_b(B_total_list[-1], K_AB_b,enzyme_AB)]
                jbp_list = [evaluate_Jbp(B_total_list[-1], K_BP,enzyme_BP)]
                jbp_b_list = [evaluate_Jbp(P_total_list[-1], K_BP_b,enzyme_BP)]
                jpx_list = [evaluate_Jpx(P_total_list[-1], K_PX,enzyme_BP)]
            elif enzyme == 'ab':
                jsa_list = [evaluate_Jsa(S_total_list[-1], K_SA, enzyme_SA)]
                jsa_b_list = [evaluate_Jsa_b(A_total_list[-1], K_SA_b, enzyme_SA)]
                jab_list = [evaluate_Jab(A_total_list[-1], K_AB, concentration)]
                jab_b_list = [evaluate_Jab_b(B_total_list[-1], K_AB_b, concentration)]
                jbp_list = [evaluate_Jbp(B_total_list[-1], K_BP, enzyme_BP)]
                jbp_b_list = [evaluate_Jbp(P_total_list[-1], K_BP_b, enzyme_BP)]
                jpx_list = [evaluate_Jpx(P_total_list[-1], K_PX, enzyme_BP)]

            elif enzyme == 'bp':
                jsa_list = [evaluate_Jsa(S_total_list[-1], K_SA, enzyme_SA)]
                jsa_b_list = [evaluate_Jsa_b(A_total_list[-1], K_SA_b, enzyme_SA)]
                jab_list = [evaluate_Jab(A_total_list[-1], K_AB, enzyme_BP)]
                jab_b_list = [evaluate_Jab_b(B_total_list[-1], K_AB_b, enzyme_BP)]
                jbp_list = [evaluate_Jbp(B_total_list[-1], K_BP, concentration)]
                jbp_b_list = [evaluate_Jbp(P_total_list[-1], K_BP_b, concentration)]
                jpx_list = [evaluate_Jpx(P_total_list[-1], K_PX, concentration)]

            for i in times:

                current_S = S
                current_LS = LS
                current_S = S
                current_LS = LS

                if enzyme == 'sa':
                    current_Jsa = evaluate_Jsa(S_total_list[-1], K_SA, concentration)
                    current_Jsa_b = evaluate_Jsa_b(A_total_list[-1], K_SA_b, concentration)
                    current_Jab = evaluate_Jab(A_total_list[-1], K_AB, enzyme_AB)
                    current_Jab_b = evaluate_Jab_b(B_total_list[-1], K_AB_b, enzyme_AB)
                    current_Jbp = evaluate_Jbp(B_total_list[-1], K_BP, enzyme_BP)
                    current_Jbp_b = evaluate_Jbp(P_total_list[-1], K_BP_b, enzyme_BP)
                    current_Jpx = evaluate_Jpx(P_total_list[-1], K_PX, enzyme_PX)

                elif enzyme == 'ab':
                    current_Jsa = evaluate_Jsa(S_total_list[-1], K_SA, enzyme_SA)
                    current_Jsa_b = evaluate_Jsa_b(A_total_list[-1], K_SA_b, enzyme_SA)
                    current_Jab = evaluate_Jab(A_total_list[-1], K_AB, concentration)
                    current_Jab_b = evaluate_Jab_b(B_total_list[-1], K_AB_b, concentration)
                    current_Jbp = evaluate_Jbp(B_total_list[-1], K_BP, enzyme_BP)
                    current_Jbp_b = evaluate_Jbp(P_total_list[-1], K_BP_b, enzyme_BP)
                    current_Jpx = evaluate_Jpx(P_total_list[-1], K_PX, enzyme_PX)

                elif enzyme == 'bp':
                    current_Jsa = evaluate_Jsa(S_total_list[-1], K_SA, enzyme_SA)
                    current_Jsa_b = evaluate_Jsa_b(A_total_list[-1], K_SA_b, enzyme_SA)
                    current_Jab = evaluate_Jab(A_total_list[-1], K_AB, enzyme_AB)
                    current_Jab_b = evaluate_Jab_b(B_total_list[-1], K_AB_b, enzyme_AB)
                    current_Jbp = evaluate_Jbp(B_total_list[-1], K_BP, concentration)
                    current_Jbp_b = evaluate_Jbp(P_total_list[-1], K_BP_b, concentration)
                    current_Jpx = evaluate_Jpx(P_total_list[-1], K_PX, enzyme_PX)
                    
                elif enzyme == 'px':
                    current_Jsa = evaluate_Jsa(S_total_list[-1], K_SA, enzyme_SA)
                    current_Jsa_b = evaluate_Jsa_b(A_total_list[-1], K_SA_b, enzyme_SA)
                    current_Jab = evaluate_Jab(A_total_list[-1], K_AB, enzyme_AB)
                    current_Jab_b = evaluate_Jab_b(B_total_list[-1], K_AB_b, enzyme_AB)
                    current_Jbp = evaluate_Jbp(B_total_list[-1], K_BP, enzyme_BP)
                    current_Jbp_b = evaluate_Jbp(P_total_list[-1], K_BP_b, enzyme_BP)
                    current_Jpx = evaluate_Jpx(P_total_list[-1], K_PX, concentration)

                current_A = A_list[-1] + (evaluate_dLAdt(current_Jsa, current_Jsa_b, current_Jab, current_Jab_b, (1 - fs_list[-1]), (1 - fa_list[-1]), (1 - fb_list[-1])) * timestep)
                current_B = B_list[-1] + (evaluate_dLBdt(current_Jab, current_Jab_b, current_Jbp, current_Jbp_b, (1 - fa_list[-1]), (1 - fb_list[-1]), (1 - fp_list[-1])) * timestep)
                current_P = P_list[-1] + (evaluate_dLPdt(current_Jbp, current_Jbp_b, current_Jpx, (1 - fb_list[-1]), (1 - fp_list[-1])) * timestep)

                current_LA = LA_list[-1] + (evaluate_dLAdt(current_Jsa, current_Jsa_b, current_Jab, current_Jab_b, fs_list[-1], fa_list[-1], fb_list[-1]) * timestep)
                current_LB = LB_list[-1] + (evaluate_dLBdt(current_Jab, current_Jab_b, current_Jbp, current_Jbp_b, fa_list[-1], fb_list[-1], fp_list[-1]) * timestep)
                current_LP = LP_list[-1] + (evaluate_dLPdt(current_Jbp, current_Jbp_b, current_Jpx, fb_list[-1], fp_list[-1]) * timestep)

                current_S_total = current_S + current_LS
                current_A_total = current_A + current_LA
                current_B_total = current_B + current_LB
                current_P_total = current_P + current_LP

                current_fs = evaluate_fractional_labeling(current_LS, current_S_total)
                current_fa = evaluate_fractional_labeling(current_LA, current_A_total)
                current_fb = evaluate_fractional_labeling(current_LB, current_B_total)
                current_fp = evaluate_fractional_labeling(current_LP, current_P_total)

                LS_list.append(current_LS)
                LA_list.append(current_LA)
                LB_list.append(current_LB)
                LP_list.append(current_LP)

                S_list.append(current_S)
                A_list.append(current_A)
                B_list.append(current_B)
                P_list.append(current_P)

                S_total_list.append(current_S_total)
                A_total_list.append(current_A_total)
                B_total_list.append(current_B_total)
                P_total_list.append(current_P_total)

                fs_list.append(current_fs)
                fa_list.append(current_fa)
                fb_list.append(current_fb)
                fp_list.append(current_fp)

                jsa_list.append(current_Jsa)
                jsa_b_list.append(current_Jsa_b)
                jab_list.append(current_Jab)
                jab_b_list.append(current_Jab_b)
                jbp_list.append(current_Jbp)
                jbp_b_list.append(current_Jbp_b)
                jpx_list.append(current_Jpx)
                
            jbp_values.append(jpx_list[-1])
            
        spline = interpolate.splrep(np.log(concentrations), np.log(jbp_values))
        dydx = interpolate.splev(np.log(concentrations), spline, der=1)
        fluxes_to_plot.append(np.log(jbp_values))
        dydx_to_plot.append(dydx)


    fig, ax = plt.subplots(1, 4,figsize=(20, 5))
    
    focus_index_jsa = np.where(np.array(concentrations).round(3) == enzyme_SA)
    focus_index_jab = np.where(np.array(concentrations).round(3) == enzyme_AB)
    focus_index_jbp = np.where(np.array(concentrations).round(3) == enzyme_BP)
    focus_index_jpx = np.where(np.array(concentrations).round(3) == enzyme_PX)

    control_coefficient_jsa = str(round(float(dydx_to_plot[0][focus_index_jsa]), 4))
    control_coefficient_jab = str(round(float(dydx_to_plot[1][focus_index_jab]), 4))
    control_coefficient_jbp = str(round(float(dydx_to_plot[2][focus_index_jbp]), 4))
    control_coefficient_jpx = str(round(float(dydx_to_plot[3][focus_index_jpx]), 4))
    
    ax[0].plot(np.log(concentrations), fluxes_to_plot[0], label = 'ln Flux')
    ax[0].plot(np.log(concentrations), dydx_to_plot[0], label = '1st Derivative')
    y_jsa = (dydx_to_plot[0][focus_index_jsa] * (np.log(concentrations) - np.log(concentrations[focus_index_jsa]))) + fluxes_to_plot[0][focus_index_jsa]
    ax[0].plot(np.log(concentrations), y_jsa, label='Tangent Line')
    ax[0].set_title('S -> A Control Coefficient: {0}'.format(control_coefficient_jsa, 4), fontweight = 'bold')
    ax[0].legend()
    ax[0].set_xlabel('Ln Concentration', fontweight = 'bold')
    ax[0].set_ylabel('Ln Flux', fontweight = 'bold')
    ax[0].vlines(np.log(concentrations[focus_index_jsa]), -3, 3)

    
    ax[1].plot(np.log(concentrations), fluxes_to_plot[1], label = 'ln Flux')
    y_jab = (dydx_to_plot[1][focus_index_jab] * (np.log(concentrations) - np.log(concentrations[focus_index_jab]))) + fluxes_to_plot[1][focus_index_jab]
    ax[1].plot(np.log(concentrations),y_jab,label='Tangent Line')
    ax[1].set_title('A -> B Control Coefficient: {0}'.format(control_coefficient_jab, 4), fontweight = 'bold')
    ax[1].legend()
    ax[1].set_xlabel('Ln Concentration', fontweight = 'bold')
    ax[1].set_ylabel('Ln Flux', fontweight = 'bold')
    ax[1].vlines(np.log(concentrations[focus_index_jab]), -3, 0.5)

    
    ax[2].plot(np.log(concentrations), fluxes_to_plot[2], label = 'ln Flux')
    ax[2].plot(np.log(concentrations), dydx_to_plot[2], label = '1st Derivative')
    y_jbp = (dydx_to_plot[2][focus_index_jbp] * (np.log(concentrations) - np.log(concentrations[focus_index_jbp]))) + fluxes_to_plot[2][focus_index_jbp]
    ax[2].plot(np.log(concentrations), y_jbp, label = 'Tangent Line')
    ax[2].set_title('B -> P Control Coefficient: {0}'.format(control_coefficient_jbp, 4), fontweight = 'bold')
    ax[2].legend()
    ax[2].set_xlabel('Ln Concentration', fontweight = 'bold')
    ax[2].set_ylabel('Ln Flux', fontweight = 'bold')
    ax[2].vlines(np.log(concentrations[focus_index_jbp]), -3, 3)

    ax[3].plot(np.log(concentrations), fluxes_to_plot[3], label = 'ln Flux')
    ax[3].plot(np.log(concentrations), dydx_to_plot[3], label = '1st Derivative')
    y_jpx = (dydx_to_plot[3][focus_index_jpx] * (np.log(concentrations) - np.log(concentrations[focus_index_jpx]))) + fluxes_to_plot[2][focus_index_jbp]
    ax[3].plot(np.log(concentrations), y_jpx, label = 'Tangent Line')
    ax[3].set_title('P -> X Control Coefficient: {0}'.format(control_coefficient_jpx, 4), fontweight = 'bold')
    ax[3].legend()
    ax[3].set_xlabel('Ln Concentration', fontweight = 'bold')
    ax[3].set_ylabel('Ln Flux', fontweight = 'bold')
    ax[3].vlines(np.log(concentrations[focus_index_jpx]), -3, 3)

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions we need for our interface. 
#----------------------------------------------------------------------------------------------------------------

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from ipywidgets import GridspecLayout
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
%matplotlib inline


def interactive_element_MCA():
    total_time_slider = widgets.IntSlider(min = 1, max = 10000, step = 1, value = 10000)
    timestep_slider = widgets.FloatSlider(min= 0.01, max = 1, step = 0.01, value = 1)
    S_slider = widgets.IntSlider(min = 0, max = 100, step = 1, value = 0)
    A_slider = widgets.IntSlider(min = 0, max = 100, step = 1, value = 0)
    B_slider = widgets.IntSlider(min = 0, max = 100, step = 1, value = 0)
    P_slider = widgets.IntSlider(min = 0, max = 100, step = 1, value = 0)
    LS_slider = widgets.IntSlider(min = 0, max = 100, step = 1, value = 100)
    LA_slider = widgets.IntSlider(min = 0, max = 100, step = 1, value = 0)
    LB_slider = widgets.IntSlider(min = 0, max = 100, step = 1, value = 0)
    LP_slider = widgets.IntSlider(min = 0, max = 100, step = 1, value = 0)
    k_SA_slider = widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = 0.1)
    k_SA_b_slider = widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = 0.09)
    k_AB_slider = widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = 0.1)
    k_AB_b_slider = widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = 0.01)
    k_BP_slider = widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = 0.1)
    k_BP_b_slider = widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = 0.01)
    k_PX_slider = widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = 0.01)
    enzyme_SA_slider = widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = 0.3)
    enzyme_AB_slider = widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = 0.1)
    enzyme_BP_slider = widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = 0.1)


    button = widgets.Button(description='Run')

    def on_button_clicked(_): 
        with out:
            clear_output()
            auto_MCA_interactive()
            show_inline_matplotlib_plots()

    button.on_click(on_button_clicked) 

    def auto_MCA_interactive(): 
        total_time = total_time_slider.value
        timestep = timestep_slider.value
        S = S_slider.value
        A = A_slider.value
        B = B_slider.value
        P = P_slider.value
        LS = LS_slider.value
        LA = LA_slider.value
        LB = LB_slider.value
        LP = LP_slider.value
        k_SA = k_SA_slider.value
        k_SA_b = k_SA_b_slider.value
        k_AB = k_AB_slider.value
        k_AB_b = k_AB_b_slider.value
        k_BP = k_BP_slider.value
        k_BP_b = k_BP_b_slider.value
        k_PX = k_PX_slider.value
        enzyme_SA = enzyme_SA_slider.value
        enzyme_AB = enzyme_AB_slider.value
        enzyme_BP = enzyme_BP_slider.value
        
        auto_MCA(total_time, timestep S, A, B, P, LS, LA, LB,
            LP, k_SA, k_SA_b, k_AB, k_AB_b, k_BP, k_BP_b, k_PX, enzyme_SA, enzyme_AB, enzyme_BP)

    out = widgets.Output()

    grid = GridspecLayout(7, 6) 

    grid[0,0] = widgets.HTML('<b>Total Time</b>') 
    grid[0,1] = total_time_slider
    grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
    grid[0,3] = timestep_slider
    grid[0,4] = widgets.HTML('<b>Constant [S]</b>')
    grid[0,5] = S_slider
    
    grid[1,0] = widgets.HTML('<b>Initial [A]</b>')
    grid[1,1] = A_slider
    grid[1,2] = widgets.HTML('<b>Initial [B]</b>')
    grid[1,3] = B_slider
    grid[1,4] = widgets.HTML('<b>Initial [P]</b>')
    grid[1,5] = P_slider
    
    grid[2,0] = widgets.HTML('<b>Constant [LS]</b>')
    grid[2,1] = LS_slider
    grid[2,2] = widgets.HTML('<b>Initial [LA]</b>')
    grid[2,3] = LA_slider
    grid[2,4] = widgets.HTML('<b>Initial [LB]</b>')
    grid[2,5] = LB_slider
    
    grid[3,0] = widgets.HTML('<b>Initial [LP]</b>')
    grid[3,1] = LP_slider
    grid[3,2] = widgets.HTML('<b>S -> A Rate Constant</b>')
    grid[3,3] = k_SA_slider
    grid[3,4] = widgets.HTML('<b>A -> B Rate Constant</b>')
    grid[3,5] = k_AB_slider

    grid[4,0] = widgets.HTML('<b>B -> P Rate Constant</b>')
    grid[4,1] = k_BP_slider
    grid[4,2] = widgets.HTML('<b>A -> S Rate Constant</b>')
    grid[4,3] = k_SA_b_slider
    grid[4,4] = widgets.HTML('<b>B -> A Rate Constant</b>')
    grid[4,5] = k_AB_b_slider

    grid[5,0] = widgets.HTML('<b>P -> B Rate Constant</b>')
    grid[5,1] = k_BP_b_slider
    grid[5,2] = widgets.HTML('<b>P -> X Rate Constant</b>')
    grid[5,3] = k_PX_slider
    grid[5,4] = widgets.HTML('<b>[SA Enzyme]')
    grid[5,5] = enzyme_SA_slider
    
    grid[6,0] = widgets.HTML('<b>[AB Enzyme]')
    grid[6,1] = enzyme_AB_slider
    grid[6,2] = widgets.HTML('<b>[BP Enzyme]')
    grid[6,3] = enzyme_BP_slider
    grid[6,4] = button    

    display(grid,out) 

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

interactive_element_MCA()

<div class="alert alert-block alert-success">
    <b>Discussion:</b> How do the results compare with what you originally found? Could this have ramifications for the implementation of MCA in real experiments and bioengineering efforts?
</div>